<a href="https://colab.research.google.com/github/J-Nobull/Migration_Research/blob/main/Migration_Capstone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Capstone Research Project on Migration Within the USA

In [ ]:
# Setup initial environment
!pip install census
import pandas as pd
import numpy as np
import os
import requests
import time
from io import BytesIO
from census import Census

print('\nEnvironment Ready')


Environment Ready


# Define Keys
(KEY SECRETS NOT SET UP YET, USER MUST GET THEIR OWN KEY AND REPLACE Key-Here)

Get API keys:  
https://apps.bea.gov/API/signup/  
https://data.bls.gov/registrationEngine/  
https://api.census.gov/data/key_signup.html

In [ ]:
# from getpass import getpass
'''
def get_api_key(name):
    key = os.getenv(name)
    if not key:
        key = getpass(f"Enter {name} (hidden input): ")
    return key

API_KEY_BEA = get_api_key('API_KEY_BEA')
API_KEY_BLS = get_api_key('API_KEY_BLS')
API_KEY_CENSUS = get_api_key('API_KEY_CENSUS')

print('API keys loaded')
'''

Define Helper function to create FIPS  
(common key to merge all datasets)

In [ ]:
# Function to create 5-digit FIPS codes
def create_fips(state_fips_series, county_fips_series):
    return state_fips_series.astype(str).str.zfill(2) + \
           county_fips_series.astype(str).str.zfill(3)

#Import all datasets


##Import from Bureau of Economic Analysis (1 of 8)  
The following constant variables will be dropped from the respective files:  
PARPP-3: 'CL_UNIT' & 'UNIT_MULT'  
MARPP-3: 'CL_UNIT' & 'UNIT_MULT'  
CAINC1-3: 'CL_UNIT': 'Dollars', 'UNIT_MULT' & 'NoteRef'; as well as  'GeoName'  
CAGDP1-1: 'CL_UNIT': 'Thousands of chained 2017 dollars' & 'UNIT_MULT'; as well as 'GeoName'

In [ ]:
API_KEY_BEA = 'Key-Here'
BEA_URL = 'https://apps.bea.gov/api/data'
YEARS = list(range(2011, 2022))

# Define Tables with LineCodes and GeoFips
TABLES = [
    {'name': 'PARPP', 'linecode': '3', # Cost of living for Metro/Non-metro
     'geofips': 'PORT', 'desc': 'RPP_portion', 'filename': 'BEA_rpp_p.csv'},
    {'name': 'MARPP', 'linecode': '3', # Cost of living for Urban areas (MSAs)
     'geofips': 'MSA', 'desc': 'RPP_msa', 'filename': 'BEA_rpp_m.csv'},
    {'name': 'CAINC1', 'linecode': '3', # County Per Capita Income (PCI)
     'geofips': 'COUNTY', 'desc': 'BEA_pci', 'filename': 'BEA_PCI.csv'},
    {'name': 'CAGDP1', 'linecode': '1', # County Gross Domestic Product (GDP)
     'geofips': 'COUNTY', 'desc': 'BEA_gdp', 'filename': 'BEA_GDP.csv'}]

# Fetch tables
print('Downloading BEA data (2011-2021)...')

for table in TABLES:

    print(f"\nFetching {table['desc']} ({table['name']})...")

    params = {
        'UserID': API_KEY_BEA,
        'method': 'GetData',
        'datasetname': 'Regional',
        'TableName': table['name'],
        'LineCode': table['linecode'],
        'Year': YEARS,
        'GeoFips': table['geofips'],
        'ResultFormat': 'json'}

    response = requests.get(BEA_URL, params=params, timeout=120)
    data = response.json()

# Show errors
    if 'Error' in data.get('BEAAPI', {}):
        print(f" ❌ Error: {data['BEAAPI']['Error']['Detail']}")
        continue

    df = pd.DataFrame(data['BEAAPI']['Results']['Data'])

# Initialize rename_map dynamically based on available columns
    rename_map = {}

# Rename 'TimePeriod' to 'YEAR'
    if 'TimePeriod' in df.columns: rename_map['TimePeriod'] = 'YEAR'

# Rename 'DataValue' to table['desc']
    if 'DataValue' in df.columns: rename_map['DataValue'] = table['desc']
    else:
# If 'DataValue' not present, manually create column with NaNs
        print(f"  Warning: 'DataValue' column missing for {table['name']}. Creating '{table['desc']}' with NaN.")
        df[table['desc']] = np.nan

# Conditionally rename 'GeoFips'
    if 'GeoFips' in df.columns:
        if table['name'] in ['CAINC1', 'CAGDP1']:
            rename_map['GeoFips'] = 'FIPS'
        else: # For PARPP and MARPP
            rename_map['GeoFips'] = 'GeoFIPS' # lowercase to uppercase

# Apply renaming
    df = df.rename(columns=rename_map)

# After renaming, this handles errors.
    if 'YEAR' not in df.columns:
        df['YEAR'] = np.nan
        print(f"  Warning: 'TimePeriod' or 'YEAR' column not found in {table['name']}. 'YEAR' created with NaN.")
    df['YEAR'] = pd.to_numeric(df['YEAR'], errors='coerce')

# Convert the 'desc' column to numeric
    if table['desc'] in df.columns:
        df[table['desc']] = pd.to_numeric(df[table['desc']], errors='coerce')

# Keep 'GeoName' and other variables based on table type
    if table['name'] in ['PARPP', 'MARPP']:
# Ensure 'GeoName' is present before trying to select it
        cols_to_keep = ['GeoFIPS', 'YEAR', table['desc']]
        if 'GeoName' in df.columns:
            cols_to_keep.insert(1, 'GeoName') # Insert GeoName after GeoFips
        df = df[cols_to_keep]
    else: # For CAINC1 and CAGDP1
        # Ensure 'FIPS' column exists after renaming
        fips_col = 'FIPS'
        if fips_col not in df.columns:
            print(f"  Warning: '{fips_col}' column not found after renaming for {table['name']}. Skipping selection.")
            continue # Skip saving if critical column is missing
        df = df[[fips_col, 'YEAR', table['desc']]]

# Save to CSV
    df.to_csv(table['filename'], index=False)

    print(f"   Saved {len(df):,} rows to {table['filename']}")
# Add check to confirm file existence immediately after saving
    if os.path.exists(table['filename']):
        print(f"    (Confirmed: {table['filename']} exists on disk)")
    else:
        print(f"    (ERROR: {table['filename']} NOT found on disk after saving!)")
    print(df.head())
    time.sleep(2)

print('\n' + '='*30)
print('BEA IMPORT COMPLETE')
print('='*30)
print('Files created:')
for table in TABLES:
    print(f"  - {table['filename']}")

In [ ]:
# Join BEA_PCI and BEA_GDP on FIPS and Year
BEA_PCI = pd.read_csv('BEA_PCI.csv')
BEA_PCI['FIPS'] = BEA_PCI['FIPS'].astype(str).str.zfill(5)
BEA_GDP = pd.read_csv('BEA_GDP.csv')
BEA_GDP['FIPS'] = BEA_GDP['FIPS'].astype(str).str.zfill(5)

BEA_join1 = pd.merge(BEA_PCI, BEA_GDP, on=['FIPS', 'YEAR'], how='inner')

# Create state variable from first 2 digits of FIPS
BEA_join1['STATE'] = BEA_join1[
    'FIPS'].str[:2].fillna(0).astype(int).astype(str).str.zfill(2)

# Display
print(BEA_join1.head())

In [ ]:
# Download CBSA delineation file to convert RPP_MSA to FIPS-State 2-digit
url = 'https://www2.census.gov/programs-surveys/metro-micro/geographies/reference-files/2013/delineation-files/list1.xls'

# CBSA file will be used to convert MSA into 5-digit FIPS
print("Downloading Feb 2013 CBSA delineation file...")
response = requests.get(url)
cbsa_file = pd.read_excel(BytesIO(response.content), skiprows=2)

print(f"Loaded {len(cbsa_file):,} records from delineation file")

# Create 5-digit FIPS
cbsa_file['FIPS'] = create_fips(
    cbsa_file['FIPS State Code'].fillna(0).astype(int),
    cbsa_file['FIPS County Code'].fillna(0).astype(int))

cbsa_file.rename(columns={
    'CBSA Code': 'CBSA_code',
    'FIPS State Code': 'STATE'}, inplace=True)
cbsa_file['STATE'] = cbsa_file['STATE'].fillna(0).astype(int).astype(str).str.zfill(2)

BEA_rpp_m = pd.read_csv('BEA_rpp_m.csv')
BEA_rpp_m['GeoFIPS'] = BEA_rpp_m['GeoFIPS'].astype(str).str.zfill(5)

print(f"\nLoaded BEA file with {len(BEA_rpp_m):,} rows")
print(f"BEA GeoFIPS dtype: {BEA_rpp_m['GeoFIPS'].dtype}")
print(f"Crosswalk CBSA_code dtype: {cbsa_file['CBSA_code'].dtype}")

# Merge
df_merged = BEA_rpp_m.merge(cbsa_file, left_on='GeoFIPS', right_on='CBSA_code', how='left')

# Select final columns
BEA_join2 = df_merged[['FIPS', 'YEAR', 'RPP_msa', 'STATE']]

# Display
print(BEA_join2.head())

Line 22 clears unneeded columns.  
To visually verify full.merge and .fillna completed cleanly, create docstring: delete # on lines 21 & 23.

In [ ]:
BEA_rpp_p = pd.read_csv('BEA_rpp_p.csv', dtype={'GeoFIPS': str})
# Create state variable from first 2 digits of FIPS
BEA_rpp_p['FIPS'] = BEA_rpp_p['GeoFIPS'].astype(str)
BEA_rpp_p['STATE'] = BEA_rpp_p['FIPS'].str[:2]
BEA_rpp_p = BEA_rpp_p[BEA_rpp_p['FIPS'].str.endswith('999')].copy()

# Merge BEA_join1 (PCI + GDP) with BEA_join2 (MSA/metro RPP)
BEA_full = BEA_join1.merge(BEA_join2[['FIPS', 'YEAR', 'RPP_msa']],
                            on=['FIPS', 'YEAR'],
                            how='left')

# Merge state-level RPP for filling NaN values
BEA_import = BEA_full.merge(BEA_rpp_p[['STATE', 'YEAR', 'RPP_portion']],
                          on=['STATE', 'YEAR'],
                          how='left')

# Fill null RPP values with RPP_portion
BEA_import['RPP'] = BEA_import['RPP_msa'].fillna(BEA_import['RPP_portion'])

# Drop the unneeded columns
# '''
BEA_import.drop(columns=['RPP_msa', 'RPP_portion', 'STATE'], inplace=True)
# '''
print(f"BEA_import complete with {len(BEA_import):,} rows")
print(f"RPP values filled: {BEA_import['RPP'].notna().sum():,}")

BEA_import.to_csv('BEA_import.csv', index=False)

# Display
print(BEA_import.head(35))

## Import from Census Bureau (2 of 8)  
import selected variables from Census "Detailed Tables"

In [ ]:
API_KEY_CENSUS = 'Key-Here'
CENSUS_URL = 'https://api.census.gov/data'
YEARS = list(range(2011, 2022))

# ACS variable definitions
ACS_VARS = {
# Population
    'B01003_001E': 'total_population',
# Age
    'B01002_001E': 'median_age',
# Housing
    'B25003_001E': 'housing_total',
    'B25003_002E': 'owner_occupied',
    'B25003_003E': 'renter_occupied',
    'B25002_003E': 'vacant',
    'B25077_001E': 'median_home_value',
# Households
    'B11001_002E': 'family_households',
# Marital Status
    'B12001_001E': 'marital_total',
    'B12001_003E': 'never_married_male',
    'B12001_004E': 'now_married_male',
    'B12001_010E': 'divorced_male',
    'B12001_009E': 'widowed_male',
    'B12001_012E': 'never_married_female',
    'B12001_013E': 'now_married_female',
    'B12001_018E': 'widowed_female',
    'B12001_019E': 'divorced_female',
# Children (all under 18)
    'B09001_002E': 'under_18_in_hh',
# Race/Ethnicity (sums to total)
    'B03002_003E': 'white',
    'B03002_004E': 'black',
    'B03002_005E': 'native',
    'B03002_006E': 'asian',
    'B03002_007E': 'pacific_islander',
    'B03002_008E': 'other_race',
    'B03002_009E': 'two_or_more_nh',
    'B03002_012E': 'hispanic',
# Education
    'B15002_001E': 'education_total_sex',
    'B15002_011E': 'male_complete_hs',
    'B15002_012E': 'male_some_college<1',
    'B15002_013E': 'male_some_college>1',
    'B15002_014E': 'male_associates',
    'B15002_015E': 'male_bachelors',
    'B15002_016E': 'male_masters',
    'B15002_017E': 'male_professional',
    'B15002_018E': 'male_doctorate',
    'B15002_028E': 'female_complete_hs',
    'B15002_029E': 'female_some_college<1',
    'B15002_030E': 'female_some_college>1',
    'B15002_031E': 'female_associates',
    'B15002_032E': 'female_bachelors',
    'B15002_033E': 'female_masters',
    'B15002_034E': 'female_professional',
    'B15002_035E': 'female_doctorate',
# Income
    'B19013_001E': 'median_hh_income',
# Employment
    'B23025_004E': 'employed',
    'B23025_005E': 'unemployed',
    'B23025_007E': 'not_in_labor_force',
# Commute Time
    'B08303_002E': 'commute_less_5min',
    'B08303_003E': 'commute_5_9min',
    'B08303_004E': 'commute_10_14min',
    'B08303_005E': 'commute_15_19min',
    'B08303_006E': 'commute_20_24min',
    'B08303_007E': 'commute_25_29min',
    'B08303_008E': 'commute_30_34min',
    'B08303_009E': 'commute_35_39min',
    'B08303_010E': 'commute_40_44min',
    'B08303_011E': 'commute_45_59min',
    'B08303_012E': 'commute_60_89min',
    'B08303_013E': 'commute_90_plus_min',
# Worked from home
    'B08137_020E': 'work_in_owned_home',
    'B08137_021E': 'work_in_rental',
# Estate taxes paid
    'B25103_001E': 'median_property_taxes',
# Industry
    'C24060_001E': 'occupation_total',
    'C24060_002E': 'Mgmt_Biz_Sci_Arts',
    'C24060_003E': 'Services',
    'C24060_004E': 'Sales_Admin',
    'C24060_005E': 'Nat-rsrc_Constr_Maint',
    'C24060_006E': 'Prod_Transp_Mvng'}

def fetch_census_batch(year, variables):
    # Fetch one batch of Census variables for all counties.
    var_list = ','.join(variables)
    params = {
        'get': var_list,
        'for': 'county:*',
        'key': API_KEY_CENSUS}

    url = f'{CENSUS_URL}/{year}/acs/acs5'
    response = requests.get(url, params=params, timeout=120)

    # Check for HTTP errors first
    if response.status_code != 200:
        print(f"❌ HTTP Error for {year}: Status Code {response.status_code}")
        print(f"Response content: {response.text}")
        return pd.DataFrame()

    try:
        data = response.json()
    except requests.exceptions.JSONDecodeError:
        print(f"❌JSON Decode Error in {year}: Could not parse response as JSON")
        print(f"Response content: {response.text}")
        return pd.DataFrame()

    # Show errors based on parsed JSON (if any)
    if 'error' in data or len(data) <= 1:
        print(f"❌ Error: No data returned or API error for {year}")
        if 'error' in data:
            print(f"API error details: {data['error']}")
        return pd.DataFrame()

    df = pd.DataFrame(data[1:], columns=data[0])
    return df

# Fetch data
print('Downloading Census data (2011-2021)...\n')

all_years = []
all_vars = list(ACS_VARS.keys())
batch1 = all_vars[:45]  # Census API limit is 50 variables
batch2 = all_vars[45:]

for year in YEARS:
    print(f"Fetching year {year}...")

    # Fetch both batches
    df1 = fetch_census_batch(year, batch1)
    df2 = fetch_census_batch(year, batch2)

    if df1.empty:
        continue

    # Merge batches on state and county
    if not df2.empty:
        year_df = pd.merge(df1, df2, on=['state', 'county'], how='outer')
    else:
        year_df = df1

    # Rename variables
    year_df = year_df.rename(columns=ACS_VARS)

    # Create FIPS
    year_df['FIPS'] = create_fips(year_df['state'], year_df['county'])
    year_df['Year'] = year

    # Convert to numeric
    for col in ACS_VARS.values():
        if col in year_df.columns:
            year_df[col] = pd.to_numeric(year_df[col], errors='coerce')

    all_years.append(year_df)
    print(f"Saved {len(year_df):,} rows")
    time.sleep(0.5)

# Combine all years
if all_years:
    CEN_df = pd.concat(all_years, ignore_index=True)

    # Keep only needed columns
    keep_cols = ['FIPS', 'Year'] + [col for col in ACS_VARS.values()
                                     if col in CEN_df.columns]
    CEN_df = CEN_df[keep_cols]

    # Save to CSV
    CEN_df.to_csv('Census_import.csv', index=False)


    print("\n" + "=" * 30)
    print('CENSUS IMPORT COMPLETE')
    print("\n" + "=" * 30)
    print(f"\n --Saved {len(CEN_df):,} rows")
    print(f"   Counties: {CEN_df['FIPS'].nunique()}")
    print(f"   Years: {CEN_df['Year'].min()}-{CEN_df['Year'].max()}")
    print(f"   Variables: {len(ACS_VARS)}")
else:
    print('\n❌ Error: No data was downloaded')

## Import from Bureau of Labor Statistics (3 of 8)  
only import 'unemployment rate'

In [ ]:
API_KEY_BLS = 'Key-Here'
BLS_URL = 'https://api.bls.gov/publicAPI/v2/timeseries/data/'
YEARS = list(range(2011, 2022))

print('Building BLS series IDs from Census FIPS codes...\n')

# Get FIPS from Census data, ensure FIPS is string and zero-filled
census_df = pd.read_csv('Census_import.csv', dtype={'FIPS': str})
census_df['FIPS'] = census_df['FIPS'].str.zfill(5)
unique_fips = sorted(census_df['FIPS'].unique())

# Build series IDs for matching counties
all_series = [f'LAUCN{fips}0000000003' for fips in unique_fips]

print(f'Found {len(all_series):,} counties from Census data')
print(f'Will require {(len(all_series) + 49) // 50} batches\n')

# Download in batches of 50
batch_size = 50
all_data = []

print('Downloading BLS unemployment rate data (2011-2021)...\n')

for i in range(0, len(all_series), batch_size):
    batch = all_series[i:i+batch_size]
    batch_num = i // batch_size + 1
    total_batches = (len(all_series) + 49) // 50

    payload = {
        'seriesid': batch,
        'startyear': '2011',
        'endyear': '2021',
        'registrationkey': API_KEY_BLS,
        'annualaverage': True}

    response = requests.post(BLS_URL, json=payload, timeout=120)
    data = response.json()

# Check for errors
    if data.get('status') != 'REQUEST_SUCCEEDED':
        print(f'❌ Batch {batch_num}/{total_batches} error: {data.get("message", "Unknown")}')
        continue

# Parse response
    for series in data['Results']['series']:
        series_id = series['seriesID']
        fips = series_id[5:10]  # Extract FIPS

        for item in series['data']:
            if item['period'] == 'M13':  # Average unemployment rate for year
                value = item['value']

                # Handle missing data (represented as '-')
                if value == '-':
                    unemployment_rate = None
                else:
                    unemployment_rate = float(value)

                all_data.append({
                    'FIPS': fips,
                    'YEAR': int(item['year']),
                    'unemploy_rate': unemployment_rate})

    print(f'  Batch {batch_num}/{total_batches}')
    time.sleep(2)

# Save to CSV
if all_data:
    BLS_import = pd.DataFrame(all_data)
    BLS_import.to_csv('BLS_import.csv', index=False)

    print("\n" + "=" * 30)
    print('BLS IMPORT COMPLETE')
    print("=" * 30)
    print(f'\n Saved {len(BLS_import):,} rows')
    print(f'  Counties: {BLS_import["FIPS"].nunique()}')
    print(f'  Years: {BLS_import["YEAR"].min()}-{BLS_import["YEAR"].max()}')
    print(f'  Missing values: {BLS_import["unemploy_rate"].isna().sum()}')
else:
    print('\n❌ Error: No data downloaded')

# Display
print(BLS_import.head(25))

## Read data from IRS (4 of 8)  
download 11 State-to-state inflow files for 2011-2021 at:  
https://www.irs.gov/statistics/soi-tax-stats-migration-data

In [ ]:
# Define the years and file paths
years = list(range(2011, 2022))
year_mapping = {
    2011: 'countyinflow1112.csv',
    2012: 'countyinflow1213.csv',
    2013: 'countyinflow1314.csv',
    2014: 'countyinflow1415.csv',
    2015: 'countyinflow1516.csv',
    2016: 'countyinflow1617.csv',
    2017: 'countyinflow1718.csv',
    2018: 'countyinflow1819.csv',
    2019: 'countyinflow1920.csv',
    2020: 'countyinflow2021.csv',
    2021: 'countyinflow2122.csv'}

print("Starting IRS Migration Data Import...")

all_data = []

for year, filepath in year_mapping.items():
    print(f"Processing {year}...")

    IRS_df = pd.read_csv(filepath, encoding='latin-1')
    IRS_df['YEAR'] = year

    IRS_df['out_FIPS'] = create_fips(IRS_df['y1_statefips'], IRS_df['y1_countyfips'])
    IRS_df['in_FIPS'] = create_fips(IRS_df['y2_statefips'], IRS_df['y2_countyfips'])
# Rename n2 = Number of Individuals counted on that calendar years' returns
    IRS_df = IRS_df.rename(columns={'n2': 'Movers'})
    IRS_df.replace(-1, np.nan, inplace=True)

    print(f"  Rows: {len(IRS_df):,}")

    all_data.append(IRS_df)

IRS_full = pd.concat(all_data, ignore_index=True)

# Order variables
final_cols = ['out_FIPS', 'in_FIPS', 'YEAR', 'Movers', 'agi']

# Create the dataframe
IRS_import = IRS_full[final_cols]
IRS_import.to_csv('IRS_import.csv', index=False)


print('\n' + '='*30)
print("\n Saved to IRS_import.csv")
print('\n' + '='*30)
print(f"\nTotal rows: {len(IRS_import):,}")

# Display
IRS_import.head(10)

Starting IRS Migration Data Import...
Processing 2011...
  Rows: 130,101
Processing 2012...
  Rows: 131,931
Processing 2013...
  Rows: 86,193
Processing 2014...
  Rows: 75,527
Processing 2015...
  Rows: 86,330
Processing 2016...
  Rows: 98,874
Processing 2017...
  Rows: 87,932
Processing 2018...
  Rows: 83,762
Processing 2019...
  Rows: 87,552
Processing 2020...
  Rows: 89,850
Processing 2021...
  Rows: 90,498


 Saved to IRS_import.csv


Total rows: 1,048,550


,out_FIPS,in_FIPS,YEAR,Movers,agi
0,96000,01000,2011,238230.0,4549431.0
1,97000,01000,2011,235901.0,4500247.0
2,97001,01000,2011,135124.0,2381712.0
3,97003,01000,2011,100777.0,2118535.0
4,98000,01000,2011,2329.0,49184.0
5,96000,01001,2011,4618.0,83494.0
6,97000,01001,2011,4511.0,80882.0
7,97001,01001,2011,2961.0,48904.0
8,97003,01001,2011,1550.0,31978.0
9,98000,01001,2011,107.0,2612.0


## Read 4 datafiles from USDA (8 of 8)  
download 4 files (names are top line in each cell) from:  
https://www.ers.usda.gov/data-products

In [ ]:
# 5: County Typology Codes, 2015 Edition
# --------------------------------------------------
typology = pd.read_csv('erscountytypology2015edition.csv')

# Ensure FIPS is 5-digit string
typology['FIPStxt'] = typology['FIPStxt'].astype(str).str.zfill(5)

# Rename columns
typology.rename(columns={
    'FIPStxt': 'FIPS',
    'Economic Types Type_2015_Update non-overlapping': 'Industry_type',
    'Farming_2015_Update': 'Farming',
    'Mining_2015-Update': 'Mining',
    'Manufacturing_2015_Update': 'Mfging',
    'Government_2015_Update': 'Govt',
    'Recreation_2015_Update': 'Rec',
    'Nonspecialized_2015_Update': 'Nonspec',
    'Low_Education_2015_Update': 'Low_Ed_cnty',
    'Low_Employment_Cnty_2008_2012_25_64': 'Low_emp_cnty',
    'Retirement_Dest_2015_Update': 'Retire_dest_cnty',
    'Persistent_Poverty_2013': 'Persistent_Pov_cnty',
    'Persistent_Related_Child_Poverty_2013': 'Pers_chld_pov_cnty'}, inplace=True)

# Drop variables to avoid duplication
typology = typology.drop(columns=['State', 'County_name',
    'Metro-nonmetro status, 2013 0=Nonmetro 1=Metro',
    'Economic_Type_Label'])

print(f"Economic Typology 2015: {len(typology):,} counties")
# Display
typology.head(10)

In [ ]:
# 6: Natural Amenities Scale
# --------------------------------------------------
# Data starts at row 105
amenities = pd.read_excel('natamenf_1_.xls', skiprows=104, engine='xlrd')

# Ensure FIPS is 5-digit string
amenities['FIPS Code'] = amenities['FIPS Code'].astype(str).str.zfill(5)

# Rename variables
amenities.rename(columns={
    'FIPS Code': 'FIPS',
    'Scale': 'Amenity_score',
    ' 1=Low  7=High': 'Amenity_rank'}, inplace=True)

# Select only Amenity variables, drop the component variables
amenities = amenities[['FIPS', 'Amenity_score', 'Amenity_rank']]

print(f"Natural Amenities: {len(amenities):,} counties")
# Display
amenities.head(10)

In [ ]:
# 7: Rural-Urban Continuum Codes (RUCC) 2013
# --------------------------------------------------
rucc2013 = pd.read_excel('ruralurbancodes2013.xls')

# Ensure FIPS is 5-digit string
rucc2013['FIPS'] = rucc2013['FIPS'].astype(str).str.zfill(5)

# Rename Population variable
rucc2013 = rucc2013.rename(columns={'Population_2010': 'POP_2010'})

# Select only RUCC code and population (drop 3 variables)
rucc2013 = rucc2013.drop(columns=['Description', 'State', 'County_Name'])

print(f"RUCC 2013: {len(rucc2013):,} counties")
# Display
rucc2013.head(10)

In [ ]:
# 8: Rural-Urban Continuum Codes (RUCC) 2023
# --------------------------------------------------
# Data is long format - 3 rows (Population, RUCC code, Description)
ruccode2023 = pd.read_csv('Ruralurbancontinuumcodes2023.csv', encoding='latin-1')

# Pivot from long to wide
rucc2023 = ruccode2023.pivot(
    index='FIPS',
    columns='Attribute',
    values='Value')

# Reset the index to turn pivoted index into a regular column
rucc2023 = rucc2023.reset_index()

# Clear the columns name attribute after pivoting
rucc2023.columns.name = None

# Ensure FIPS is 5-digit string
rucc2023['FIPS'] = rucc2023['FIPS'].astype(str).str.zfill(5)

# Rename Population variable
rucc2023 = rucc2023.rename(columns={'Population_2020': 'POP_2020'})

# Select only RUCC code and population (drop description)
rucc2023 = rucc2023[['FIPS', 'POP_2020', 'RUCC_2023']]

print(f"RUCC 2023: {len(rucc2023):,} counties")
# Display
rucc2023.head(10)

Merge all USDA data together

In [ ]:
# Merge All USDA and Create County-Year Panel, with RUCC2013 as base
usda_base = rucc2013[['FIPS', 'POP_2010', 'RUCC_2013']].copy()

# Merge all classification variables
usda_base = usda_base.merge(rucc2023, on='FIPS', how='left')
usda_base = usda_base.merge(amenities, on='FIPS', how='left')
usda_base = usda_base.merge(typology, on='FIPS', how='left')

print(f"\nMerged USDA classifications: {len(usda_base):,} counties")
print(f"Total variables: {len(usda_base.columns)}")

# Expand to county-year panel (2011-2021)
usda_panel = []
for year in range(2011, 2022):
    df_year = usda_base.copy()
    df_year['YEAR'] = year
# Move YEAR to second column
    cols = df_year.columns.tolist()
    cols = [cols[0], 'YEAR'] + [c for c in cols[1:] if c != 'YEAR']
    df_year = df_year[cols]
    usda_panel.append(df_year)

usda_import = pd.concat(usda_panel, ignore_index=True)

# Save single consolidated panel
usda_import.to_csv('USDA_import.csv', index=False)

print('\n' + '='*30)
print("USDA MERGE COMPLETE")
print('\n' + '='*30)
print(f"\nTotal county-year observations: {len(usda_import):,}")
print(f"Unique counties: {usda_import['FIPS'].nunique():,}")
print(f"Total variables: {len(usda_import.columns)}")
# Display
usda_import.head(10)

# Inspect and Clean datasets

In [ ]:
BEA_clean = pd.read_csv('BEA_import.csv')

# Convert 'FIPS' to 5-digit string
BEA_clean['FIPS'] = BEA_clean['FIPS'].astype(str).str.zfill(5)

print('\nBEA first rows')
print(BEA_clean.head())
print('\nBEA variable info')
print(BEA_clean.info())
print('\nBEA descriptive stats')
print(BEA_clean.describe())
print('\n BEA nulls')
print(BEA_clean.isnull().sum())


BEA first rows
    FIPS  YEAR  BEA_pci  BEA_gdp     RPP
0  01001  2020    45068  1746979  87.517
1  01001  2021    49174  1736001  88.497
2  01001  2011    34430  1493906  91.098
3  01001  2012    35151  1726577  93.269
4  01001  2013    35348  1618151  91.394

BEA variable info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34298 entries, 0 to 34297
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   FIPS     34298 non-null  object 
 1   YEAR     34298 non-null  int64  
 2   BEA_pci  34298 non-null  int64  
 3   BEA_gdp  34298 non-null  int64  
 4   RPP      34298 non-null  float64
dtypes: float64(1), int64(3), object(1)
memory usage: 1.3+ MB
None

BEA descriptive stats
               YEAR        BEA_pci       BEA_gdp           RPP
count  34298.000000   34298.000000  3.429800e+04  34298.000000
mean    2016.000000   42470.220188  6.115655e+06     90.968153
std        3.162324   12990.093071  2.665519e+07      6.529244
min

No initial BEA issues to clean

In [ ]:
BLS_clean = pd.read_csv('BLS_import.csv')

# Convert 'FIPS' to 5-digit string
BLS_clean['FIPS'] = BLS_clean['FIPS'].astype(str).str.zfill(5)

print('\nBLS first rows')
print(BLS_clean.head())
print('\nBLS variable info')
print(BLS_clean.info())
print('\nBLS descriptive stats')
print(BLS_clean.describe())
print('\nBLS nulls')
print(BLS_clean.isnull().sum())


BLS first rows
    FIPS  YEAR  unemploy_rate
0  01001  2021            2.7
1  01001  2020            5.3
2  01001  2019            2.9
3  01001  2018            3.6
4  01001  2017            4.0

BLS variable info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35323 entries, 0 to 35322
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   FIPS           35323 non-null  object 
 1   YEAR           35323 non-null  int64  
 2   unemploy_rate  35245 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 828.0+ KB
None

BLS descriptive stats
               YEAR  unemploy_rate
count  35323.000000   35245.000000
mean    2016.000255       6.104962
std        3.162416       2.960603
min     2011.000000       1.100000
25%     2013.000000       4.000000
50%     2016.000000       5.400000
75%     2019.000000       7.500000
max     2021.000000      28.900000

BLS nulls
FIPS              0
YEAR              0

Inspect BLS  
78 MVs
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~  
Puerto Rico is missing 2020 values
Drop Puerto Rico (72000 Series)

In [ ]:
# Exclude Puerto Rico (FIPS codes starting with '72')
BLS_clean = BLS_clean[~BLS_clean['FIPS'].astype(str).str.startswith('72')]

print('\nBLS missing values after cleaning:')
print(BLS_clean.isnull().sum())
print(BLS_clean.tail())
print(BLS_clean.info())


BLS missing values after cleaning:
FIPS             0
YEAR             0
unemploy_rate    0
dtype: int64
        FIPS  YEAR  unemploy_rate
34460  56045  2015            3.4
34461  56045  2014            3.5
34462  56045  2013            3.6
34463  56045  2012            4.1
34464  56045  2011            4.8
<class 'pandas.core.frame.DataFrame'>
Index: 34465 entries, 0 to 34464
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   FIPS           34465 non-null  object 
 1   YEAR           34465 non-null  int64  
 2   unemploy_rate  34465 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 1.1+ MB
None


In [ ]:
Census_clean = pd.read_csv('Census_import.csv')

# Convert 'FIPS' to 5-digit string
Census_clean['FIPS'] = Census_clean['FIPS'].astype(str).str.zfill(5)

# Exclude Puerto Rico
Census_clean = Census_clean[
    ~Census_clean['FIPS'].astype(str).str.startswith('72')]

print('\nCensus first rows:')
print(Census_clean.head())
pd.set_option('display.max_columns', None) # Display ALL variable info/stats
print('\nCensus variable info:')
print(Census_clean.info())
print('\nCensus descriptive stats:')
print(Census_clean.describe())
print('\nCensus nulls:')
print(Census_clean.isnull().sum())
pd.reset_option('display.max_columns') # Reset display to default

Inspect Census  
inspect 23 vars with 1MV  
inspect 1 var with 2 MVs  
inspect 1 var with 11 MVs  
inspect med_home_value -666666666 values  
inspect med_hh_income -666666666 values  
inspect med_prop_taxes -666666666 values  
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Kalawao, HI(15005): DROP ALL  
total_population averages 105 from 2011-2021.  
BEA.gov: "Kalawao County, Hawaii is combined with Maui County.  
Separate estimates for the jurisdictions making up the combination areas are not available"  
(0.002% of Maui population)

Rio Arriba, New Mexico(35039):  
23 MVs in 2018: Fill with average of 2017 and 2019.  

Buffalo, South Dakota(46017) NaN's:  
2020 med_hm_value=56680, filled with average 2011-2021.  
4 med_prop_taxes=634, filled with average 2011-2021.  

Mellette, South Dakota(46095) NaN's:  
2020 med_hm_value=41900, 2019 med_prop_taxes=1144, 2020 med_prop_taxes=1369   
filled with average of year before and year after.  

Jeff Davis, Texas(48243) NaN's:  
2020 median_hh_income=50232, filled with average of 2011-2019.  
2020 median_property_taxes=2296, filled with average of 2019 and 2021.

Kenedy, Texas(48261) NaN's 2015-2021:  
2014 med_hm_value appears to be an error [166700],  
4 NaN and 2014 replaced with average of 2011-2019=40150.  

Loving, TX(48301) --  
*2015 median_hh_income=61250, fill with average of 2014-2016.  
2021 median_hh_income=66064, fill with average of 2011-2020.
*median_home_value: Utilized data from 2 nearest counties Reeves(48389) and Winkler(48495):  
imputed values for 2016-2021 = 101750, 119100, 131700, 128300, 169400, 178700.  
*median_property_taxes: Utillized data from Reeves and Winkler to impute 2015-2021 values:  
(significantly lower than state median) = 1350, 1480, 1568, 1960, 2156, 2105, 2200.  


In [ ]:
# Drop Kalawao, HI (FIPS code 15005)
Census_clean = Census_clean[~Census_clean['FIPS'].astype(str).str.startswith('15005')]

for col in Census_import.select_dtypes(include=['float64', 'int64']).columns:
    if col != 'YEAR': # Exclude 'YEAR' from imputation as it's a year, not a feature to be averaged
        Census_import[col].fillna(Census_import[col].mean(), inplace=True)

# Ensure 'YEAR' is an integer type (already int64, but explicitly setting as per instructions)
Census_import['YEAR'] = Census_import['YEAR'].astype(int)

print('\nCensus_import after cleaning:')
print(Census_import.isnull().sum())
print(Census_import.info())


In [ ]:
USDA_clean.to_csv('USDA_test.csv', index=False)

In [ ]:
USDA_clean = pd.read_csv('USDA_import.csv')

# Convert 'FIPS' to 5-digit string
USDA_clean['FIPS'] = USDA_clean['FIPS'].astype(str).str.zfill(5)

# Exclude Puerto Rico and territories; FIPS codes greater than '56999'
USDA_cleant = USDA_clean[USDA_clean['FIPS'].astype(int) <= 56999]

print('\nUSDA descriptive stats:')
print(USDA_clean.describe())
print('\nUSDA nulls:')
print(USDA_clean.isnull().sum())


USDA first rows:
    FIPS  YEAR  POP_2010  RUCC_2013  POP_2020  RUCC_2023  Amenity_score  \
0  01001  2011     54571        2.0   58805.0        2.0           0.78   
1  01003  2011    182265        3.0  231767.0        3.0           1.82   
2  01005  2011     27457        6.0   25223.0        6.0           0.19   
3  01007  2011     22915        1.0   22293.0        1.0          -0.15   
4  01009  2011     57322        1.0   59134.0        1.0           0.23   

   Amenity_rank  Industry_type  Farming  ...  Mfging  Govt  Rec  Nonspec  \
0           4.0            0.0      0.0  ...     0.0   0.0  0.0      1.0   
1           4.0            5.0      0.0  ...     0.0   0.0  1.0      0.0   
2           4.0            3.0      0.0  ...     1.0   0.0  0.0      0.0   
3           3.0            0.0      0.0  ...     0.0   0.0  0.0      1.0   
4           4.0            0.0      0.0  ...     0.0   0.0  0.0      1.0   

   Low_Ed_cnty  Low_emp_cnty  Pop_Loss_2010  Retire_dest_cnty  \
0        

Inspect USDA  
Amenity_score & Amenity_rank  MV=396  
POP_2020 & RUCC_2023          MV=132  
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Alaska and Hawaii are not part of USDA Amenities Scale  
[fill score=0 (average) and rank=0 (outside 1-7 ranking)]  

Not sure what to do with these:  
09xxx CT 8 counties became 9 planning regions RUCC_2023 is the only different database with POP=3605944 and codes=1,2,1,2,4,4,2,2,2(almost identical) impute 2010_pop into 2020 * 1.00891, keep RUCC codes, drop(09110-09190)
46113 renamed 46102 in 2015 (make 46108?)
51515 (Bedford city) folded into 51019 (Bedford County) in 2014, incorporate RUCC_2010 and census values into 51019

In [ ]:
# Mask Alaska and Hawaii
alaska_fips_mask = USDA_clean['FIPS'].astype(str).str.startswith('02')
hawaii_fips_mask = USDA_clean['FIPS'].astype(str).str.startswith('15')
ak_hi_mask = alaska_fips_mask | hawaii_fips_mask

# Fill Alaska & Hawaii Amenities with 0's
USDA_clean.loc[ak_hi_mask, 'Amenity_score'] = USDA_clean.loc[
    ak_hi_mask, 'Amenity_score'].fillna(0)
USDA_clean.loc[ak_hi_mask, 'Amenity_rank'] = USDA_clean.loc[
    ak_hi_mask, 'Amenity_rank'].fillna(0)

print(f"AK/HI null values:\n{USDA_clean[ak_hi_mask][[ \
        'FIPS', 'Amenity_score', 'Amenity_rank']].isnull().sum()}")
print(f"\n{USDA_clean[['POP_2020', 'RUCC_2023']].isnull().sum()}")
'''
# Impute missing 2020s data with 2010s data
USDA_clean['POP_2020'] = USDA_clean[
    'POP_2020'].fillna(USDA_clean['POP_2010'])
USDA_clean['RUCC_2023'] = USDA_clean[
    'RUCC_2023'].fillna(USDA_clean['RUCC_2013'])


'''

AK/HI null values:
FIPS             0
Amenity_score    0
Amenity_rank     0
dtype: int64


'\n# Impute missing 2020s data with 2010s data\nUSDA_clean[\'POP_2020\'] = USDA_clean[\n    \'POP_2020\'].fillna(USDA_clean[\'POP_2010\'])\nUSDA_clean[\'RUCC_2023\'] = USDA_clean[\n    \'RUCC_2023\'].fillna(USDA_clean[\'RUCC_2013\'])\n\nprint(f"\nNull values after imputation:\n{USDA_clean[[\'POP_2020\', \'RUCC_2023\']].isnull().sum()}")\n'

In [ ]:
print(USDA_clean.info())

In [ ]:
02xxx AK has no amenity scale/rank fill with 0/2
09xxx CT 8 counties became 9 planning regions
46113 renamed 46102 in 2015
51515 incorporated into 51019 between 2013-2023

IRS_import**  
Ensure str zfill (5)  
FIPS that are an issue:  
xx000 = state totals, drop  
15005, drop  

origin_fips:  
57000-98000, drop

In [ ]:
IRS_trim = pd.read_csv('IRS_import.csv')

# Convert to integer for filtering
IRS_trim['FIPS_int'] = pd.to_numeric(IRS_trim['in_FIPS'], errors='coerce')
IRS_trim['out_FIPS_int'] = pd.to_numeric(IRS_trim['out_FIPS'], errors='coerce')
# Drop territories/foreign origins (57000-98000)
foreign_origin = (IRS_trim['out_FIPS_int'] >= 57000)
# Drop state totals (xx000)
state_totals_dest = (IRS_trim['FIPS_int'] % 1000 == 0)
state_totals_origin = (IRS_trim['out_FIPS_int'] % 1000 == 0)
# Drop non-movers (same origin and destination)
non_movers = (IRS_trim['out_FIPS_int'] == IRS_trim['FIPS_int'])
# Drop Kalawao, HI (15005)
kalawao = (IRS_trim['FIPS_int'] == 15005)
# Combined drop mask
drop_mask = (state_totals_dest | state_totals_origin |
             kalawao | foreign_origin | non_movers)
IRS_clean = IRS_trim[~drop_mask].copy()

IRS_clean['in_FIPS'] = IRS_clean[
    'FIPS_int'].astype(int).astype(str).str.zfill(5)
IRS_clean['out_FIPS'] = IRS_clean[
    'out_FIPS_int'].astype(int).astype(str).str.zfill(5)
IRS_clean = IRS_clean.drop(columns=['FIPS_int', 'out_FIPS_int'])

IRS_clean.to_csv('IRS_clean.csv', index=False)

print('\nIRS variable info:')
print(IRS_clean.info())
print('\nIRS descriptive stats:')
print(IRS_clean.describe())


IRS variable info:
<class 'pandas.core.frame.DataFrame'>
Index: 627182 entries, 11 to 1048532
Data columns (total 5 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   out_FIPS  627182 non-null  object 
 1   in_FIPS   627182 non-null  object 
 2   YEAR      627182 non-null  int64  
 3   Movers    627182 non-null  float64
 4   agi       627180 non-null  float64
dtypes: float64(2), int64(1), object(2)
memory usage: 28.7+ MB
None

IRS descriptive stats:
                YEAR         Movers           agi
count  627182.000000  627182.000000  6.271800e+05
mean     2015.545720     194.794084  7.033679e+03
std         3.366377     798.687576  3.465237e+04
min      2011.000000      10.000000 -1.283400e+06
25%      2012.000000      41.000000  1.070000e+03
50%      2016.000000      65.000000  1.976000e+03
75%      2019.000000     130.000000  4.263000e+03
max      2021.000000   49355.000000  3.015639e+06


create 2 IRS datafiles:  
one to add to panel with 'net_movers' and keep one county-to-county for deeper trend analysis.

In [ ]:
# Step 3: Calculate net migration per county-year
inflows = IRS_clean.groupby(['in_FIPS', 'YEAR']).agg({
    'Movers': 'sum',
    'agi': 'sum'}).rename(columns={'Movers': 'move_in', 'agi': 'agi_in'})
outflows = IRS_clean.groupby(['out_FIPS', 'YEAR']).agg({
    'Movers': 'sum',
    'agi': 'sum'}).rename(columns={'Movers': 'move_out', 'agi': 'agi_out'})
outflows.index.names = ['FIPS', 'YEAR']

# Merge and calculate net
IRS_panel = inflows.join(outflows, how='outer').fillna(0).reset_index()
IRS_panel['net_movers'] = IRS_panel['move_in'] - IRS_panel['move_out']
IRS_panel['net_agi'] = IRS_panel['agi_in'] - IRS_panel['agi_out']

# Reorder columns
cols = ['FIPS', 'YEAR', 'move_in', 'move_out', 'net_movers', 'agi_in', 'agi_out', 'net_agi']
IRS_panel = IRS_panel[cols]

print(f"  Panel rows: {len(IRS_panel):,}")
print(f"  Unique counties: {IRS_panel['FIPS'].nunique():,}")
print(f"  Years: {sorted(IRS_panel['YEAR'].unique())}")

# Step 4: Save outputs
print("\n--- Step 4: Save outputs ---")

IRS_panel.to_csv('IRS_panel.csv', index=False)
print(f"  ✓ Saved IRS_panel.csv ({len(IRS_panel):,} rows)")


In [ ]:
print(IRS_clean.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 627182 entries, 0 to 627181
Data columns (total 5 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   out_FIPS  627182 non-null  int64  
 1   in_FIPS   627182 non-null  int64  
 2   YEAR      627182 non-null  int64  
 3   Movers    627182 non-null  float64
 4   agi       627180 non-null  float64
dtypes: float64(2), int64(3)
memory usage: 23.9 MB
None


In [ ]:
IRS_clean = pd.read_csv('IRS_clean.csv')
IRS_dyadic = IRS_clean[['out_FIPS', 'in_FIPS', 'YEAR', 'Movers', 'agi']].copy()
IRS_dyadic = IRS_dyadic.rename(columns={
    'out_FIPS': 'origin_fips',
    'in_FIPS': 'dest_fips',
    'Movers': 'n_migrants'})
IRS_dyadic.to_csv('IRS_dyadic.csv', index=False)
print(f"  ✓ Saved IRS_dyadic.csv ({len(IRS_dyadic):,} rows)")

print(f"\nIRS Dyadic preview:\n{IRS_dyadic.head()}")

  ✓ Saved IRS_dyadic.csv (627,182 rows)

IRS Dyadic preview:
   origin_fips  dest_fips  YEAR  n_migrants      agi
0         1051       1001  2011      1016.0  18398.0
1         1101       1001  2011       982.0  15955.0
2         1021       1001  2011       192.0   2228.0
3         1047       1001  2011       126.0   1638.0
4         1073       1001  2011       104.0   1585.0


In [ ]:
# Inspect IRS_import

⚛⚛⚛⚛⚛⚛⚛⚛⚛⚛⚛⚛⚛⚛⚛⚛⚛⚛⚛⚛⚛⚛⚛⚛⚛⚛⚛⚛⚛⚛⚛⚛⚛⚛  
All Below is from a different project, here for coding reference

## Clean DHC data
All 322 features will need:  
to be renamed (for clarity) or  
to be dropped (for redundency)
Project will prioritize 'percent' variables, scale is improved over 'count'.

In [ ]:
# Create new working dataframe
DHC_clean = DHC_import.copy()

# Remove Puerto Rico, rows where GEOID is US72000 or greater
DHC_clean = DHC_clean[~DHC_clean['Geography'].str.startswith('0500000US72')]

# Rename Geography and Geographic Area Name
DHC_clean = DHC_clean.rename(columns={
    'Geography': 'GEOID',
    'Geographic Area Name': 'County'})
DHC_clean['GEOID'] = DHC_clean['GEOID'].str[-5:]

# Confirm
print(DHC_clean.info())

## Transform DHC data

Create new 'Under 18' and '18-19' age groups

In [ ]:
# Create new working dataframe
DHC_transform = DHC_clean.copy()

# Calculate 'Under 18' by subtracting '18 years and over' from 'totals'
DHC_transform['Total_U18'] = DHC_transform[
    'Pop_total'] - DHC_transform['Total_18+']
DHC_transform['Male_U18'] = DHC_transform[
    'Male_total'] - DHC_transform['Male_18+']
DHC_transform['Female_U18'] = DHC_transform[
    'Female_total'] - DHC_transform['Female_18+']

# Calculate 'Total_18-19' by adding all ages 0-19 and subtracting U18
DHC_transform['Total_18_19'] = (DHC_transform['Total_U5'] +
    DHC_transform['Total_5_9'] + DHC_transform['Total_10_14'] +
    DHC_transform['Total_15_19'] - DHC_transform['Total_U18'])

# Repeat for Male 18-19
DHC_transform['Male_18_19'] = (DHC_transform['Male_U5'] +
    DHC_transform['Male_5_9'] + DHC_transform['Male_10_14'] +
    DHC_transform['Male_15_19'] - DHC_transform['Male_U18'])

# Repeat for Female 18-19
DHC_transform['Female_18_19'] = (DHC_transform['Female_U5'] +
    DHC_transform['Female_5_9'] + DHC_transform['Female_10_14'] +
    DHC_transform['Female_15_19'] - DHC_transform['Female_U18'])

# Calculate '%_18-19' by dividing by 'totals'
DHC_transform['%TOTAL_18_19'] = (
    DHC_transform['Total_18_19'] / DHC_transform['Pop_total']* 100).round(2)
DHC_transform['%MALE_18_19'] = (
    DHC_transform['Male_18_19'] / DHC_transform['Male_total']* 100).round(2)
DHC_transform['%FEMALE_18_19'] = (
    DHC_transform['Female_18_19'] / DHC_transform['Female_total']* 100).round(2)

# Can now drop these columns
columns_tform_drop = [
    'Total_U5', 'Male_U5', 'Female_U5',
    'Total_5_9', 'Male_5_9', 'Female_5_9',
    'Total_10_14', 'Male_10_14', 'Female_10_14',
    'Total_15_19', 'Male_15_19', 'Female_15_19',
    'Total_18+', 'Male_18+', 'Female_18+',
    'Total_U18', 'Male_U18', 'Female_U18',
    'Total_18_19', 'Male_18_19', 'Female_18_19']
DHC_transform.drop(columns=columns_tform_drop, inplace=True)

# Reorder columns to move '18-19' before '20-24'
cols = DHC_transform.columns.tolist()
cols.insert(8, cols.pop(cols.index('%TOTAL_18_19')))
cols.insert(23, cols.pop(cols.index('%MALE_18_19')))
cols.insert(38, cols.pop(cols.index('%FEMALE_18_19')))
DHC_transform = DHC_transform[cols]

#Confirm
#pd.set_option('display.max_columns', None)
#print(DHC_transform.head())
#print(DHC_transform.info())

## Save DHC cleaned data

In [ ]:
# Create the tidy dataframe
DHC_tidy = DHC_transform.copy()

DHC_tidy.to_csv('DHC_tidy.csv', index=False)

# Import P2 data (2 of 3)

P2 data is the population living in urban or rural (PUR) areas within each county **[number of households also available (H2)]**.  
For the 2020 Census, an urban area will comprise a densely settled core of census blocks that meet minimum population density requirements; which includes adjacent territory containing non-residential urban land uses. To qualify as an urban area, the territory identified according to criteria must have a population of at least 5,000. *(Note on Alaska: See accompanying 'Alaska County' amalgamation file on github for method used to match census area to state senate district. PUR datafile combines 30 census areas into 14 'County_fips' created for this analysis)*. Also dropped Kalawao County, Hawaii: 82 rural residents, none of them voted, dropping will align it with MEDSL file when Kalawao county_fips (15005) is cleaned from MEDSL data.    
https://data.census.gov/all?q=urban+and+rural&g=010XX00US$0500000

In [ ]:
# Import next dataset
PUR_import = pd.read_csv(
    'DECENNIALDHC2020.P2-AKfix.csv', header=1)

# Inspect
print(PUR_import.info())
print(PUR_import.head())

## Clean PUR data

In [ ]:
# Create new working dataframe
PUR_clean = PUR_import.copy()

# Remove Puerto Rico: rows where GEOID is US72000 or greater
PUR_clean = PUR_clean[~PUR_clean['Geography'].str.startswith('0500000US72')]

# Rename variables to keep and drop remaining
PUR_clean = PUR_clean.rename(columns={
    'Geography': 'GEOID',
    ' !!Total:': 'Pop_total',
    ' !!Total:!!Urban': 'Pop_Urban',
    ' !!Total:!!Rural': 'Pop_Rural'})

PUR_clean['GEOID'] = PUR_clean['GEOID'].str[-5:]

# Calculate Urban percent
PUR_clean['%Urban_pop'] = (
    (PUR_clean['Pop_Urban'] / PUR_clean['Pop_total']) * 100).round(2)

# Drop the specified columns
columns_PUR_drop = ['Pop_total',
                    'Geographic Area Name',
                    ' !!Total:!!Not defined for this file']
PUR_clean.drop(columns=columns_PUR_drop, inplace=True)

# Confirm
print(PUR_clean.info())

## Save PUR data

In [ ]:
# Create the tidy dataframe
PUR_tidy = PUR_clean.copy()

PUR_tidy.to_csv('PUR_tidy.csv', index=False)

# Import MEDSL data (3 of 3)
2020 general election results for most* (46) of the 50 states and D.C. downloaded from MEDSL (the MIT Election Data and Science Lab) https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/NT66Z3  

* ALASKA: voting data is not gathered by county, MEDSL 'county_fips' is empty. Used https://www.elections.alaska.gov/results/20GENR/Map/ Votes aggregated to state senate districts (1 - 40). See accompanying 'Alaska County' amalgamation file on github for method used to match 30 census areas to 40 state senate districts. MEDSL datafile uses 14 County_fips created for this analysis. Datafile only has the 4 variables that will be utilized here.

* INDIANA: MEDSL missing multiple county results. Used https://indianavoters.in.gov/ENRHistorical/ElectionResults  Datafile only has the 4 variables that will be utilized here, aggregated to the county level  

* NEW MEXICO: To protect the privacy of voters, New Mexico 'masks' vote totals in precinct results for candidates with small vote tallies. Used https://electionstats.sos.nm.gov/contest/13250  Datafile only has the 4 variables that will be utilized here, aggregated to the county level  

* NEVADA: To protect the privacy of voters, Nevada 'masks' vote totals in precinct results for candidates with 1-10 vote tallies. Used https://www.nvsos.gov/SOSelectionPages/results/2020StateWideGeneral/ElectionSummary.aspx  Datafile only has the 4 variables that will be utilized here, aggregated to the county level

##Pre-import processing Notes:  
The below adjustments were made to the MEDSL datafiles to standardize cleaning and processing.   

1. HAWAII: Adjusted DHC and PUR data regarding Kalawao County, Hawaii. Both have fips 15005, but there are no official votes cast, removed so all files align  

1. MAINE: Uniformed and Overseas Citizens Absentee Voting tallied seperately in 23000 fips, 23000 deleted to match DHC and PUR with votes added to 23005 (most populous county)  

1. MICHIGAN: MEDSL precinct data contains precinct '9999', which are 'statistical adjustments' rows. There were minor corrections needed to match official results at https://www.michigan.gov/sos/elections/election-results-and-data/candidate-listings-and-election-results-by-county  

1. MINNESOTA: 'DEMOCRATIC FARMER LABOR' party changed to 'DEMOCRAT'  

1. MISSOURI: MEDSL tallied Kansas City votes seperately in 36000 fips. Utillized https://www.sos.mo.gov/CMSImages/ElectionResultsStatistics/November3_2020GeneralElection.pdf to aportion some votes to Jackson County with remainder assigned to Clay County (official results not available on https://www.voteclaycountymo.gov/election-results), but totals match State official numbers  

1.  NEW YORK: 'CONSERVATIVE' party changed to 'REPUBLICAN'  
'WORKING FAMILIES' party changed to 'DEMOCRAT'  

1.  NORTH DAKOTA: 'DEMOCRATIC-NPL' party changed to 'DEMOCRAT' and 'county_fips' for OGLALA LAKOTA County changed from 46113 to 46102 to match data from DHC and PUR  

1.  OREGON: Sherman County included cadidate 'BALLOTS CAST' which totaled all votes in each precinct: Deleted  

1.  PENNSYLVANIA: 1 blank 'party_detailed' vote cast for Trump, party changed to 'REPUBLICAN'  

1.  VERMONT: 3 blank 'party_detailed' votes cast for Trump, party changed to 'REPUBLICAN'  
6 blank 'party_detailed' votes cast for Biden, party changed to 'DEMOCRAT'  

##Post-import cleaning Notes:
1.  All blanks in 'party_detailed' have been verified as writein votes cast for 'THIRD' party candidates  

2.  In Nov 2020, there were over 50 recognized political parties in the US.  
DEM and REP ballots accounted for 96% of total votes. Third parties accounted for 1-4% of the vote in each state. 'THIRD' will combine any vote NOT for Presidents Biden or Trump.  



In [ ]:
# Define list of all 51 voter CSV files to process (50 states plus D.C.)
file_list = glb.glob('2020-*-precinct-general.csv')

# Define function to read, select features, and clean a single CSV file
def process_file(file_path):

    try:
# Specify data types, let 'votes' be float during import
        dtype_spec = {'office': str, 'county_fips': str,
                      'party_detailed': str, 'votes': float}
        df = pd.read_csv(file_path, dtype=dtype_spec, low_memory=False)

# Filter for President in 'office' to avoid counting multiple votes per person
# Only analyze US Presidential race (it has the most voter participation)
        df = df[df['office'] == 'US PRESIDENT'].copy()
        df = df[['office', 'county_fips', 'party_detailed', 'votes']]
        df = df.rename(columns={
            'county_fips': 'GEOID',
            'party_detailed': 'PARTY',
            'votes': 'VOTES'})
        return df

    except Exception as e:
        print(f'Error processing {file_path}: {e}')
        return None

In [ ]:
# Iterate through the file list, apply function, and store dataframes
all_processed_dataframes = [
    process_file(file_path) for file_path in file_list]

# Filter out any None values if errors occurred during processing
all_processed_dataframes = [
    df for df in all_processed_dataframes if df is not None]

# Concatenate all processed files into single dataframe
US_combined = pd.concat(all_processed_dataframes, ignore_index=True)

# Confirm
#print(US_combined.info())
#print(US_combined.head())

## Clean MEDSL data

In [ ]:
# Fill missing values with 'THIRD'
US_combined.loc[:, 'PARTY'] = US_combined['PARTY'].fillna('THIRD')

# Create list of parties to rename
print(sorted(US_combined['PARTY'].unique()))

In [ ]:
# Define other parties to replace with 'THIRD' (remove DEMOCRAT and REPUBLICAN from 'US_combined' output)
other_parties = [
    'ALLIANCE', 'ALLIANCE PARTY', 'AMERICAN', 'AMERICAN CONSTITUTION', 'AMERICAN SHOPPING', 'AMERICAN SOLIDARITY', 'APPROVAL VOTING', 'BECOMING ONE NATION', 'BIRTHDAY', 'BLANK', 'BOILING FROG', 'BREAD AND ROSES', 'BULL MOOSE', 'C.U.P', 'CONSTITUTION', 'CONSTITUTION PARTY', 'CUP', 'FREEDOM AND PROSPERITY', 'GENEALOGY KNOW YOUR FAMILY HISTORY', 'GREEN', 'GREEN INDEPENDENT', 'GREEN-RAINBOW', 'GRUMPY OLD PATRIOTS', 'INDEPENDENCE', 'INDEPENDENCE ALLIANCE', 'INDEPENDENT', 'INDEPENDENT AMERICAN', 'LIBERTARIAN', 'LIBERTY UNION', 'LIFE', 'LIFE LIBERTY CONSTITUTION', 'NATURAL LAW PARTY', 'NONE', 'NONPARTISAN', 'OREGON PROGRESSIVE', 'OTHER', 'PACIFIC GREEN', 'PARTY FOR SOCIALISM AND LIBERATION', 'PROGRESSIVE', 'PROHIBITION', 'REFORM', 'SOCIALISM', 'SOCIALISM AND LIBERATION', 'SOCIALIST', 'SOCIALIST EQUALITY', 'SOCIALIST WORKERS', 'STATEWIDE GREEN', 'UNAFFILIATED', 'UNITY', 'UNITY AMERICA', 'UNITY OF COLORADO', 'US TAXPAYERS PARTY']

# Replace these other parties with 'THIRD'
US_combined['PARTY'] = US_combined['PARTY'].replace(other_parties, 'THIRD')

# Tally Presidential votes
PRES_votes = (US_combined.groupby('PARTY', as_index=False)['VOTES']
    .sum().sort_values(by='VOTES', ascending=False))

# Confirm
print(US_combined.info())
print(US_combined['PARTY'].unique())
print(US_combined['PARTY'].value_counts(dropna=False))
print(PRES_votes)

In [ ]:
# Pivot to get vote counts by Party
US_transform = (US_combined.groupby(['GEOID', 'PARTY'])['VOTES']
    .sum()
    .unstack(fill_value=0)
    .reset_index())

# Rename columns that were the party names after unstacking
US_transform = US_transform.rename(columns={
    'DEMOCRAT': 'DEM_VOTES',
    'REPUBLICAN': 'REP_VOTES',
    'THIRD': 'THRD_VOTES'})

# Change vote columns to int32
vote_cols = ['DEM_VOTES', 'REP_VOTES', 'THRD_VOTES']
US_transform[vote_cols] = US_transform[vote_cols].astype('int32')

# Confirm
#print(US_transform.info())
#print(US_transform.head())

## Create share of vote feature

In [ ]:
# Create new working dataframe
US_tranfm2 = US_transform.copy()

# Compute TOTAL_VOTES, drop any where the sum of all votes = 0
US_tranfm2['TOTAL_VOTES'] = US_tranfm2[vote_cols].sum(axis=1).astype('int32')
US_tranfm2 = US_tranfm2[US_tranfm2['TOTAL_VOTES'] != 0]

# Compute shares of votes
US_tranfm2['DEM_SHARE'] = (
    (US_tranfm2['DEM_VOTES'] / US_tranfm2['TOTAL_VOTES'])* 100).round(2)
US_tranfm2['REP_SHARE'] = (
    (US_tranfm2['REP_VOTES'] / US_tranfm2['TOTAL_VOTES'])* 100).round(2)
US_tranfm2['THRD_SHARE'] = (
    (US_tranfm2['THRD_VOTES'] / US_tranfm2['TOTAL_VOTES'])* 100).round(2)

# Confirm
#print(US_tranfm2.info())
#print(US_tranfm2.head())

## Create political leaning feature

In [ ]:
# View values of DEM_SHARE, ensure all >0
print(sorted(US_tranfm2['DEM_SHARE'].unique()))

In [ ]:
# Create new working dataframe
US_tranfm3 = US_tranfm2.copy()

# Define the political leaning function
def determine_win(row):
# Only DEM or REP win, only consider their shares for determining lead
    shares = {
        'DEM': row['DEM_SHARE'],
        'REP': row['REP_SHARE']}

    # Determine the winning party between DEM and REP
    if shares['DEM'] > shares['REP']:
        party_win = 1 # Democrat wins = Positive lead for DEM
        party_lead = (shares['DEM'] - shares['REP']) / 100
    elif shares['REP'] > shares['DEM']: # Corrected from else
        party_win = 0 # Republican wins = Negative lead for REP
        party_lead = (shares['DEM'] - shares['REP']) / 100
    else: # Tie (very unlikely)
        party_win = 2
        party_lead = 0.0

    return party_win, round(party_lead, 2)

# Apply function and create two new variables
US_tranfm3[['PARTY_WIN', 'PARTY_LEAD']] = US_tranfm3.apply(
    determine_win, axis=1).apply(pd.Series)

# Convert 'PARTY_WIN' to int
US_tranfm3['PARTY_WIN'] = US_tranfm3['PARTY_WIN'].astype('int32')

# Confirm
print(US_tranfm3.head(10))

## Save MEDSL data

In [ ]:
# Order variables
final_cols = ['GEOID', 'TOTAL_VOTES',
              'DEM_VOTES', 'DEM_SHARE',
              'REP_VOTES', 'REP_SHARE',
              'THRD_VOTES', 'THRD_SHARE',
              'PARTY_WIN', 'PARTY_LEAD']

# Create the tidy dataframe
MEDSL_tidy = US_tranfm3[final_cols]

MEDSL_tidy.to_csv('MEDSL_tidy.csv', index=False)

# Confirm
print(MEDSL_tidy.info())

# Merge data files

# Import _tidy files here if you do not want to clean the data

In [ ]:
import pandas as pd
# Import here if utilizing the _tidy files
DHC_tidy = pd.read_csv('DHC_tidy.csv')
PUR_tidy = pd.read_csv('PUR_tidy.csv')
MEDSL_tidy = pd.read_csv('MEDSL_tidy.csv')

# Confirm
print(DHC_tidy.info())
print(PUR_tidy.info())
print(MEDSL_tidy.info())

In [ ]:
# Merge first two files
TWO_join = pd.merge(DHC_tidy, PUR_tidy, on='GEOID', how='outer')

# Confirm
#print(TWO_join.info())

In [ ]:
# Merge with third dataset
FULL_DF = pd.merge(TWO_join, MEDSL_tidy, on='GEOID', how='outer')

# change GEOID type
FULL_DF['GEOID'] = FULL_DF['GEOID'].astype(str)

# Confirm
#print(FULL_DF.info())

In [ ]:
# Create new working dataframe
FULL_transform = FULL_DF.copy()

# Split 'Name' into 'County' and 'State'
FULL_transform[['County', 'State']] = FULL_transform[
    'County'].str.split(', ', expand=True)

# Reorder columns to move 'State' to index 1
cols = FULL_transform.columns.tolist()
cols.insert(1, cols.pop(cols.index('State')))
MERGED_DF = FULL_transform[cols]

## Save MERGED datafile

In [ ]:
MERGED_DF.to_csv('MERGED_DF.csv', index=False)

# Exploratory Data Analysis (with MERGED_DF)

In [ ]:
# Setup exploration environment
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
%matplotlib inline
print('Environment Ready')

In [ ]:
MERGED_DF = pd.read_csv('MERGED_DF.csv')
# ensure GEOID is an object
MERGED_DF['GEOID'] = MERGED_DF['GEOID'].astype(str)

# Confirm
print(MERGED_DF.info())

## Basic Information

In [ ]:
# Display descriptive statistics for numerical columns
print('Number of rows:', MERGED_DF.shape[0], '(Number of counties)')
print('Number of columns:', MERGED_DF.shape[1])
print('\nMissing Values: None')
print(MERGED_DF.isna().sum().sort_values(ascending=False))

print('\nDescriptive Statistics for Numerical Columns:')
display(MERGED_DF.describe())

# Display value counts for categorical column (PARTY_WIN)
print('\nValue Counts for 'PARTY_WIN' \n0: Republican Win\n1: Democrat Win:')
display(MERGED_DF['PARTY_WIN'].value_counts())

## Visualizations

In [ ]:
# Visualize the distribution of the target variable 'PARTY_WIN'
plt.figure(figsize=(6, 4))
sns.countplot(x='PARTY_WIN', data=MERGED_DF)
plt.title('Distribution of PARTY_WIN (0: Republican Win, 1: Democrat Win)')
plt.xlabel('Party Win')
plt.ylabel('Count')
plt.xticks([0, 1], ['Republican Win', 'Democrat Win'])
plt.show()

print('')
# Visualize the distribution of 'PARTY_LEAD'
fig, axes = plt.subplots(1, 3, figsize=(15,5))
sns.histplot(MERGED_DF['DEM_SHARE'], bins=30, kde=True, ax=axes[0], color='blue')
axes[0].set_title('Democratic Vote Share')
sns.histplot(MERGED_DF['REP_SHARE'], bins=30, kde=True, ax=axes[1], color='red')
axes[1].set_title('Republican Vote Share')
sns.histplot(MERGED_DF['PARTY_LEAD'], bins=30, kde=True, ax=axes[2], color='purple')
axes[2].set_title('Margin of Victory (Party Lead)')
plt.tight_layout()
plt.show()

print('')
sns.histplot(MERGED_DF['Pop_total'], bins=50, kde=True)
plt.title('County Population Distribution')
plt.xlabel('Population')
plt.ylabel('Number of Counties')
plt.show()

print('')
sns.histplot(MERGED_DF['%Urban_pop'], bins=30, kde=True)
plt.title('Urban Population Share by County')
plt.xlabel('% Urban Population')
plt.ylabel('Number of Counties')
plt.show()

print('')
sns.histplot(MERGED_DF['MED_AGE'], bins=30, kde=True)
plt.title('Median Age Distribution')
plt.xlabel('Median Age')
plt.ylabel('Number of Counties')
plt.show()

print('')
race_cols = ['%RACE_White', '%RACE_Black', '%RACE_Latino', '%RACE_Asian']
MERGED_DF[race_cols].plot(kind='box', figsize=(8,6))
plt.title('Distribution of Racial Composition by County')
plt.ylabel('Percentage')
plt.show()

print('')
sns.scatterplot(x='%OWN_HOME', y='%RENT_HOME', data=MERGED_DF)
plt.title('Own vs Rent in Counties')
plt.xlabel('% Own Home')
plt.ylabel('% Rent Home')
plt.show()

print('')
sns.scatterplot(x='%Urban_pop', y='DEM_SHARE', data=MERGED_DF, alpha=0.6)
plt.title('Urban Population vs Democratic Vote Share')
plt.show()

print('')
sns.scatterplot(x='%RACE_White', y='REP_SHARE', data=MERGED_DF, alpha=0.6, color='red')
plt.title('% White Population vs Republican Vote Share')
plt.show()

print('')
sns.scatterplot(x='MED_AGE', y='REP_SHARE', data=MERGED_DF, alpha=0.6, color='green')
plt.title('Median Age vs Republican Vote Share')
plt.show()

## Correlation checks on separated groups of features

In [ ]:
#corr_vars = ['Pop_total', 'MED_AGE', '%Urban_pop',
#             '%RACE_White', '%RACE_Black', '%RACE_Latino',
#             '%OWN_HOME', '%RENT_HOME',
#             'DEM_SHARE', 'REP_SHARE', 'PARTY_LEAD']

#corr = MERGED_DF[corr_vars].corr()
MERGED_num = MERGED_DF.select_dtypes(include=np.number)

corr = MERGED_num.corr()

plt.figure(figsize=(10,8))
sns.heatmap(corr, annot=False, fmt='.2f', cmap='coolwarm', center=0)
plt.title('Correlation Heatmap')
plt.show()

### Analyze Age data

In [ ]:
# Create new working dataframe
MERGED_trform = MERGED_DF.copy()

# Define column groups for total, male, and female age percentages
age_total_cols = [
    col for col in MERGED_trform.columns if col.startswith('%TOTAL_')]
age_male_cols  = [
    col for col in MERGED_trform.columns if col.startswith('%MALE_')]
age_female_cols = [
    col for col in MERGED_trform.columns if col.startswith('%FEMALE_')]

# Combine all percentage age columns and the target variables
features_for_age = age_total_cols + age_male_cols + age_female_cols + [
    'PARTY_WIN', 'PARTY_LEAD']

# Calculate the correlation matrix for the selected features
corr_age = MERGED_trform[features_for_age].corr()

# Select and display only the correlations with PARTY_WIN and PARTY_LEAD
corr_age_subset = corr_age[['PARTY_WIN', 'PARTY_LEAD']].loc[
    age_total_cols + age_male_cols + age_female_cols]

# Plot heatmap for better visualization of correlations
plt.figure(figsize=(10, 15)) # Adjust figure size as needed
sns.heatmap(corr_age_subset,
            cmap='seismic_r',
            annot=True, fmt='.2f',
            vmin=-1, vmax=1)
plt.title('Correlation Heatmap: Percentage Age Groups vs PARTY_WIN and PARTY_LEAD')
plt.yticks(rotation=0)
plt.show()

#### With a clear divergence around age 55, compare 2 vs 3 age groupings  
- yng, mid, old: Looks to break groups into pos, neutral (between -0.1 and 0.1), neg  
- young, older: Looks to break age groups into positive and negative only  

In [ ]:
# Define lists of age columns for young, middle (cutoff is |0.1|), and old
age_male_yng = [col for col in MERGED_trform.columns if col.startswith('%MALE_') and any(age in col for age in ['18_19', '20_24', '25_29', '30_34', '35_39'])]
age_male_mid = [col for col in MERGED_trform.columns if col.startswith('%MALE_') and any(age in col for age in ['40_44', '45_49', '50_54'])]
age_male_old = [col for col in MERGED_trform.columns if col.startswith('%MALE_') and any(age in col for age in ['55_59', '60_64', '65_69', '70_74', '75_79', '80_84', '85+'])]
age_female_yng = [col for col in MERGED_trform.columns if col.startswith('%FEMALE_') and any(age in col for age in ['18_19', '20_24', '25_29', '30_34', '35_39', '40_44'])]
age_female_mid = [col for col in MERGED_trform.columns if col.startswith('%FEMALE_') and any(age in col for age in ['45_49', '50_54'])]
age_female_old = [col for col in MERGED_trform.columns if col.startswith('%FEMALE_') and any(age in col for age in ['55_59', '60_64', '65_69', '70_74', '75_79', '80_84', '85+'])]

# Define lists of age columns for young (cutoff is 0) and older
age_male_young = [col for col in MERGED_trform.columns if col.startswith('%MALE_') and any(age in col for age in ['18_19', '20_24', '25_29', '30_34', '35_39', '40_44', '45_49'])]
age_male_older = [col for col in MERGED_trform.columns if col.startswith('%MALE_') and any(age in col for age in ['50_54', '55_59', '60_64', '65_69', '70_74', '75_79', '80_84', '85+'])]
age_female_young = [col for col in MERGED_trform.columns if col.startswith('%FEMALE_') and any(age in col for age in ['18_19', '20_24', '25_29', '30_34', '35_39', '40_44', '45_49', '50_54'])]
age_female_older = [col for col in MERGED_trform.columns if col.startswith('%FEMALE_') and any(age in col for age in ['55_59', '60_64', '65_69', '70_74', '75_79', '80_84', '85+'])]

# Calculate the new aggregated percentage age groups
MERGED_trform['%AGE_MALE_YNG'] = MERGED_trform[age_male_yng].sum(axis=1).round(2)
MERGED_trform['%AGE_MALE_MID'] = MERGED_trform[age_male_mid].sum(axis=1).round(2)
MERGED_trform['%AGE_MALE_OLD'] = MERGED_trform[age_male_old].sum(axis=1).round(2)
MERGED_trform['%AGE_MALE_YOUNG'] = MERGED_trform[age_male_young].sum(axis=1).round(2)
MERGED_trform['%AGE_MALE_OLDER'] = MERGED_trform[age_male_older].sum(axis=1).round(2)

MERGED_trform['%AGE_FEMALE_YNG'] = MERGED_trform[age_female_yng].sum(axis=1).round(2)
MERGED_trform['%AGE_FEMALE_MID'] = MERGED_trform[age_female_mid].sum(axis=1).round(2)
MERGED_trform['%AGE_FEMALE_OLD'] = MERGED_trform[age_female_old].sum(axis=1).round(2)
MERGED_trform['%AGE_FEMALE_YOUNG'] = MERGED_trform[age_female_young].sum(axis=1).round(2)
MERGED_trform['%AGE_FEMALE_OLDER'] = MERGED_trform[age_female_older].sum(axis=1).round(2)

# Confirm
print(MERGED_trform.info())
print(MERGED_trform.head())

### Analyze race groups

In [ ]:
# Define the list of race percentage columns
race_cols = [col for col in MERGED_trform.columns if col.startswith('%RACE_')]

# Combine race percentage columns and the target variables
features_for_race = race_cols + ['PARTY_WIN', 'PARTY_LEAD']

# Calculate the correlation matrix
corr_race = MERGED_trform[features_for_race].corr()

# Select and display only the correlations with PARTY_WIN and PARTY_LEAD
corr_race_subset = corr_race[['PARTY_WIN', 'PARTY_LEAD']].loc[race_cols]

# Display the correlations
print('Correlation of Race/Ethnic Group Percentages with PARTY_WIN and PARTY_LEAD:')
display(corr_race_subset.sort_values(by='PARTY_LEAD', key=abs, ascending=False))

# Plot heatmap for better visualization
plt.figure(figsize=(8, 6))
sns.heatmap(corr_race_subset, cmap='seismic_r', annot=True, fmt='.2f', vmin=-1, vmax=1)
plt.title('Correlation Heatmap: Race/Ethnic Group Percentages vs PARTY_WIN and PARTY_LEAD')
plt.yticks(rotation=0)
plt.show()

With clear racial differences, I will try two variations of race groups  
- White and Non-White  
- White, strong political lean, more neutral lean

In [ ]:
# Define lists to compare 2 groups: non-whites or with a cutoff of |0.2|
RACE_NonWhite = [col for col in MERGED_trform.columns if col.startswith('%RACE_') and any(race in col for race in ['Asian', 'Black', 'Other', 'Latino', 'Native', 'HI_PI', 'Mixed'])]
RACE_BAO = [col for col in MERGED_trform.columns if col.startswith('%RACE_') and any(race in col for race in ['Black', 'Asian', 'Other'])]
RACE_LNHM = [col for col in MERGED_trform.columns if col.startswith('%RACE_') and any(race in col for race in ['Latino', 'Native', 'HI_PI', 'Mixed'])]

# Calculate the new aggregated percentage race groups
MERGED_trform['%RACE_NonWhite'] = MERGED_trform[RACE_NonWhite].sum(axis=1).round(2)
MERGED_trform['%RACE_BAO'] = MERGED_trform[RACE_BAO].sum(axis=1).round(2)
MERGED_trform['%RACE_LNHM'] = MERGED_trform[RACE_LNHM].sum(axis=1).round(2)

### Analyze relationship groups

In [ ]:
# Filter for columns starting with '%REL_'
rel_cols = [col for col in MERGED_trform.columns if col.startswith('%REL_')]

# Calculate the correlation of these columns with PARTY_WIN and PARTY_LEAD
corr_rel = MERGED_trform[rel_cols + ['PARTY_WIN', 'PARTY_LEAD']].corr()

# Select and display only the correlations with PARTY_WIN and PARTY_LEAD
corr_rel_subset = corr_rel[['PARTY_WIN', 'PARTY_LEAD']].loc[rel_cols]

# Display the correlations
print('Correlation of Relationship Variables with PARTY_WIN and PARTY_LEAD:')
display(corr_rel_subset.sort_values(by='PARTY_LEAD', key=abs, ascending=False))

# Optional: Visualize correlations as a heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(corr_rel_subset,
            cmap='seismic_r',
            annot=True, fmt='.2f',
            vmin=-1, vmax=1)
plt.title('Correlation Heatmap: Relationship Variables vs PARTY_WIN and PARTY_LEAD')
plt.yticks(rotation=0)
plt.show()

### Analyze household groups

In [ ]:
# Filter for columns starting with '%HH_'
hh_cols = [col for col in MERGED_trform.columns if col.startswith('%HH_')]

# Add own and urban columns
hh_cols.extend(['%OWN_HOME', '%Urban_pop'])

# Correlate these columns with PARTY_WIN and PARTY_LEAD
corr_hh = MERGED_trform[hh_cols + ['PARTY_WIN', 'PARTY_LEAD']].corr()

# Select and display only the correlations with PARTY_WIN and PARTY_LEAD
corr_hh_subset = corr_hh[['PARTY_WIN', 'PARTY_LEAD']].loc[hh_cols]

# Display the correlations
print('Correlation of Household, Ownership, and Urban Variables with PARTY_WIN and PARTY_LEAD:')
display(corr_hh_subset.sort_values(by='PARTY_LEAD', key=abs, ascending=False))

# Optional: Visualize correlations as a heatmap
plt.figure(figsize=(8, 10)) # Adjusted figure size
sns.heatmap(corr_hh_subset,
            cmap='seismic_r',
            annot=True, fmt='.2f',
            vmin=-1, vmax=1)
plt.title('Correlation Heatmap: Household, Ownership, and Urban Variables vs PARTY_WIN and PARTY_LEAD')
plt.yticks(rotation=0)
plt.show()

## Save VOTE_DF

In [ ]:
# List columns to keep (drop HH_totals, only keep M_total and F_total as ref)
columns_to_keep = [
    'GEOID', 'Male_total', 'Female_total', '%AGE_MALE_YNG', '%AGE_MALE_MID', '%AGE_MALE_OLD', '%AGE_MALE_YOUNG', '%AGE_MALE_OLDER', '%AGE_FEMALE_YNG', '%AGE_FEMALE_MID', '%AGE_FEMALE_OLD', '%AGE_FEMALE_YOUNG', '%AGE_FEMALE_OLDER', '%RACE_White', '%RACE_Black', '%RACE_Latino', '%RACE_Native', '%RACE_Asian', '%RACE_HI_PI', '%RACE_Other', '%RACE_Mixed', '%RACE_NonWhite', '%RACE_BAO', '%RACE_LNHM', '%REL_OP_SEX_MAR', '%REL_OP_SEX_UNMAR', '%REL_S_SEX_MAR', '%REL_S_SEX_UNMAR', '%REL_W_RELATIVES', '%REL_NON_REL', '%REL_MALE_JAILED', '%REL_FEMALE_JAILED', '%REL_MALE_GRP_DORM', '%REL_FEMALE_GRP_DORM', '%HH_MARRIED', '%HH_MAR_W_KIDS','%HH_NOT_MAR',  '%HH_NOT_MAR_W_KIDS', '%HH_MALE_ALONE', '%HH_MALE_65+', '%HH_MALE_W_KIDS', '%HH_FEMALE_ALONE', '%HH_FEMALE_65+', '%HH_FEMALE_W_KIDS', '%OWN_HOME', '%Urban_pop', 'PARTY_WIN', 'PARTY_LEAD']

# Create VOTE dataframe
VOTE_DF = MERGED_trform[columns_to_keep].copy()

VOTE_DF.to_csv('VOTE_DF.csv', index=False)

## EDA complete; dataframe cleaned, merged, transformed, partially reduced, and ready for analysis

# Feature analysis (with VOTE_DF)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import numpy.typing as npt
import statsmodels.api as sm
from typing import Literal, Tuple, Union
from scipy.stats import shapiro, mannwhitneyu, rankdata, norm
from sklearn.linear_model import LogisticRegression, LassoCV
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, mean_squared_error, r2_score, mean_absolute_error, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFECV
from sklearn.inspection import permutation_importance, PartialDependenceDisplay
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.multivariate.manova import MANOVA
from collections import Counter
%matplotlib inline
print('Environment Ready')

# Import VOTE_DF file here for analysis of features
(Looked at feature interactions as well, but opted to keep simple as few improved modeling)

In [ ]:
VOTE_DF = pd.read_csv('VOTE_DF.csv')
# ensure GEOID is an object
VOTE_DF['GEOID'] = VOTE_DF['GEOID'].astype(str)

# Inspect
#print(VOTE_DF.info())

## Variance check

In [ ]:
# Select numerical columns
VOTE_num = VOTE_DF.select_dtypes(include=np.number)

variances = VOTE_num.var()

# Sort variances in descending order
var_sorted = variances.sort_values(ascending=True)

# Set pandas display option to show float format
pd.set_option('display.float_format', '{:.2f}'.format)

# Confirm (Consider dropping features with low variance >0.05)
print('\nFeature Variances (sorted):')
print(var_sorted.head(20))

## Compute VIF for VOTE_DF

In [ ]:
# Remove independent variables
VOTE_features = VOTE_num.drop(columns=['PARTY_WIN', 'PARTY_LEAD'])

# Add required constant
VOTE_features = sm.add_constant(VOTE_features)

# Compute Variance Inflation Factor for each feature
VOTE_VIF = pd.DataFrame()
VOTE_VIF['Feature'] = VOTE_features.columns
# Compute VIF, handling potential inf values which occur with perfect multicollinearity
VOTE_VIF['VIF'] = [variance_inflation_factor(VOTE_features.values, i) for i in range(VOTE_features.shape[1])]

# Sort by VIF in descending order for easier analysis
VOTE_VIF = VOTE_VIF.sort_values(by='VIF', ascending=False)

# Set display to show float format
pd.set_option('display.float_format', '{:.2f}'.format)

print('VIF for VOTE_DF:')
display(VOTE_VIF)

# Correlations

## Pearson Correlation Matrix

In [ ]:
# Compute Pearson correlation matrix
pearson_corr_matrix = VOTE_DF.corr(method='pearson')

# Display the correlations
print('Pearson Correlation Matrix:')
display(pearson_corr_matrix)

# Sort Pearson correlations with PARTY_WIN and PARTY_LEAD
pearson_corr_win = pearson_corr_matrix['PARTY_WIN'].sort_values(ascending=False)
pearson_corr_lead = pearson_corr_matrix['PARTY_LEAD'].sort_values(ascending=False)

## Spearman Correlation Matrix

In [ ]:
# Compute Spearman correlation matrix
spearman_corr_matrix = VOTE_DF.corr(method='spearman')

# Display the correlations
print('\nSpearman Correlation Matrix:')
display(spearman_corr_matrix)

# Sort and store Spearman correlation results
spearman_corr_win = spearman_corr_matrix['PARTY_WIN'].sort_values(ascending=False)
spearman_corr_lead = spearman_corr_matrix['PARTY_LEAD'].sort_values(ascending=False)

## Chatterjee's Correlation  
In 2020, a paper titled 'A New Coefficient of Correlation' introduced a new coefficient measure ξ (“Xi”) which measures how much the dependent variable is a function of the independent. The result equals 0 if the two variables are independent and will be closer to 1 as the relationship strengthens. Also includes some theoretical properties that allow for hypothesis testing prior to making assumptions about the data.  

Along with the article, the R package 'XICOR' was released which contains the function xicor() which calculates ξ when X and Y vectors or matrices are provided (provides p-values for hypothesis testing).

S. Chatterjee, *A New Coefficient of Correlation* (2020), Journal of the American Statistical Association.
https://doi.org/10.48550/arXiv.1909.10140

The below code is a python xicor function based on one written by Tim Sumner https://medium.com/data-science/a-new-coefficient-of-correlation-64ae4f260310

In [ ]:
# Compute Chatterjee's Correlation
def xicor(X, Y, ties='auto', return_p=True):
    np.random.seed(1)
    X = np.asarray(X)
    Y = np.asarray(Y)
    Y_sorted = Y[np.argsort(X)]
    n = len(X)

    if ties == 'auto':
        ties = len(np.unique(Y)) < n

    if ties:
        r = rankdata(Y_sorted, method='ordinal')
        l = rankdata(Y_sorted, method='max')
        xi = 1 - n * np.sum(np.abs(np.diff(r))) / (2 * np.sum(l * (n - l)))
    else:
        r = rankdata(Y_sorted, method='ordinal')
        xi = 1 - 3 * np.sum(np.abs(np.diff(r))) / (n**2 - 1)

# p-value approximation
    p_value = norm.sf(xi, scale=2/5/np.sqrt(n))

    if return_p:
        return xi, p_value
    else:
        return xi

# Define the independent and dependent variables
features = [col for col in VOTE_DF.columns if col not in [
    'PARTY_WIN', 'PARTY_LEAD',
    'Male_total', 'Female_total']]

target_win = VOTE_DF['PARTY_WIN']
target_lead = VOTE_DF['PARTY_LEAD']

# Store xicor results
xicor_results_win = {}
xicor_results_lead = {}

# Compute xicor for each feature against PARTY_WIN
for feature in features:
    x_data = VOTE_DF[feature]
    xi_stat, xi_p_value = xicor(x_data, target_win)
    xicor_results_win[feature] = {'statistic': xi_stat, 'p_value': xi_p_value}
    #print(f'{feature}: Statistic={xi_stat:.2f}, P-value={xi_p_value:.2f}')

# Compute xicor for each feature against PARTY_LEAD
for feature in features:
    x_data = VOTE_DF[feature]
    xi_stat, xi_p_value = xicor(x_data, target_lead)
    xicor_results_lead[feature] = {'statistic': xi_stat, 'p_value': xi_p_value}
    #print(f'{feature}: Statistic={xi_stat:.2f}, P-value={xi_p_value:.2f}')

# Store Chatterjee correlation results
xi_corr_win = pd.DataFrame.from_dict(xicor_results_win, orient='index')
xi_corr_lead = pd.DataFrame.from_dict(xicor_results_lead, orient='index')

## Compare Correlation Coefficients

In [ ]:
# Combine all correlation results into a single DataFrame
correlation_comparison = pd.concat([
    xi_corr_lead['statistic'].rename('Xi_Corr_LEAD'),
    xi_corr_win['statistic'].rename('Xi_Corr_WIN'),
    pearson_corr_lead,
    pearson_corr_win,
    spearman_corr_lead,
    spearman_corr_win,
], axis=1)

# Remove the target variables  if included
correlation_comparison.drop(['PARTY_WIN', 'PARTY_LEAD'], errors='ignore', inplace=True)

# Rename features
Correlation_Table = correlation_comparison.rename(columns={
    'Pearson_Corr_PARTY_LEAD': 'Pearson_LEAD',
    'Pearson_Corr_PARTY_WIN': 'Pearson_WIN',
    'Spearman_Corr_PARTY_LEAD': 'Spearman_LEAD',
    'Spearman_Corr_PARTY_WIN': 'Spearman_WIN'})

# Display all correlations
print('Comparison of Xi, Pearson, and Spearman Correlations:')
display(Correlation_Table.round(4).sort_values(by='Xi_Corr_LEAD', ascending=False))

# Statistical test (Test for normality first)  


In [ ]:
# Separate the dataframe into two groups based on PARTY_WIN
group_Republican = VOTE_num[VOTE_num['PARTY_WIN'] == 0]
group_Democrat = VOTE_num[VOTE_num['PARTY_WIN'] == 1]

features_for_norm = VOTE_num.columns.tolist()
features_for_norm.remove('PARTY_WIN')

normality_results = {}

for feature in features_for_norm:
    data1 = group_Republican[feature]
    data2 = group_Democrat[feature]

    if len(data1) > 2 and len(data2) > 2:
        stat1, p_norm1 = shapiro(data1)
        stat2, p_norm2 = shapiro(data2)

        normality_results[feature] = {
            'Rep_p': f'{p_norm1:.2f}',
            'Dem_p': f'{p_norm2:.2f}'}
    else:
        normality_results[feature] = {
            'Rep_p': None,
            'Dem_p': None}

# Convert to DataFrame
normality_df = pd.DataFrame(normality_results).T

# Confirm (Normality will be defined as above a threshhold of 0.05)
print(normality_df)

> Almost every feature is way below 0.05 in both groups: normality is violated with one exception: Will not use T-test.

## Run Mann-Whitney U Test

In [ ]:
mannwhit_results = []

for feature in features_for_norm:
    if feature == 'PARTY_LEAD':
        continue

    data1 = group_Republican[feature]
    data2 = group_Democrat[feature]

    if len(data1) < 2 or len(data2) < 2:
        continue

    U_stat, p_value = mannwhitneyu(data1, data2, alternative='two-sided')

    if p_value < 0.05:
        mannwhit_results.append({
            'Feature': feature,
            'DEM_median': data2.median(),
            'REP_median': data1.median(),
            'U_stat': U_stat,
            'p_value': p_value,
            'n_dem': len(data2),
            'n_rep': len(data1)})

mannwhit_df = pd.DataFrame(mannwhit_results)

# Derive additional stats
mannwhit_df['diff_median'] = mannwhit_df['DEM_median'] - mannwhit_df['REP_median']

mannwhit_df['R_biserial'] = 1 - (2 * mannwhit_df['U_stat'] / (
                            mannwhit_df['n_dem'] * mannwhit_df['n_rep']))

mannwhit_df['Cohens_d'] = (2 * mannwhit_df['R_biserial']
                          ) / np.sqrt(1 - mannwhit_df['R_biserial']**2)

# Add qualitative labels
def label_effect_size(d):
    d = abs(d)
    if d < 0.2:
        return 'Negligible'
    elif d < 0.5:
        return 'Small'
    elif d < 0.8:
        return 'Medium'
    else:
        return 'Large'

mannwhit_df['Effect_size'] = mannwhit_df['Cohens_d'].astype(float).apply(label_effect_size)

# Reorder columns for priority in table (consider dropping n_ features)
cols = mannwhit_df.columns.tolist()
cols.insert(3, cols.pop(cols.index('diff_median')))
cols.insert(5, cols.pop(cols.index('Cohens_d')))
cols.insert(6, cols.pop(cols.index('Effect_size')))
cols.insert(7, cols.pop(cols.index('R_biserial')))
mannwhit_df = mannwhit_df[cols]

# Format after sorting
mannwhit_df['DEM_median'] = mannwhit_df['DEM_median'].map(lambda x: f'{x:.2f}')
mannwhit_df['REP_median'] = mannwhit_df['REP_median'].map(lambda x: f'{x:.2f}')
mannwhit_df['diff_median'] = mannwhit_df['diff_median'].map(lambda x: f'{x:.2f}')
mannwhit_df['Cohens_d'] = mannwhit_df['Cohens_d'].map(lambda x: f'{x:.2f}')
mannwhit_df['R_biserial'] = mannwhit_df['R_biserial'].map(lambda x: f'{x:.2f}')
mannwhit_df['p_value'] = mannwhit_df['p_value'].map(lambda x: f'{x:.2f}')

# Confirm
display(mannwhit_df.sort_values(by='Cohens_d', ascending=False))

# Feature importance

## Feature Importance for PARTY_WIN from Logistic Regression

In [ ]:
# Define the features to exclude based on p-values
# Could drop Same_Sex features, but not ready to drop yet
features_to_exclude = ['']

# Select features for logistic regression, excluding the specified ones
features_for_logit = [col for col in VOTE_DF.columns if col not in features_to_exclude + ['GEOID', 'Male_total', 'Female_total', 'PARTY_WIN', 'PARTY_LEAD']]

X0 = VOTE_DF[features_for_logit]
y0 = VOTE_DF['PARTY_WIN']

# Split data into training and testing sets (recommended for model evaluation)
X0_train, X0_test, y0_train, y0_test = train_test_split(
    X0, y0, test_size=0.2, random_state=1,
    stratify=y0) # To maintain class distribution

# Standardize the features
scaler = StandardScaler()
X0_train_scaled = scaler.fit_transform(X0_train)
X0_test_scaled = scaler.transform(X0_test)

# Initialize and train the Logistic Regression model with regularization
# Using default L2 penalty and balanced class weight
logit_model_sklearn = LogisticRegression(
    random_state=1,
    class_weight='balanced',
    max_iter=1000) # Increased max_iter for convergence
logit_model_sklearn.fit(X0_train_scaled, y0_train)

# Confirm feature importances from the trained model (coefficients)
print('Feature Importance (Coefficients from Regularized Logistic Regression):')
logit_feature_importance = pd.Series(
    logit_model_sklearn.coef_[0], index=features_for_logit)
print(logit_feature_importance.sort_values(ascending=False))

# Plot feature importances
plt.figure(figsize=(10, 8)) # Adjusted figure size for better readability
logit_feature_importance.sort_values().plot(kind='barh')
plt.title('Feature Importance from Logistic Regression(WIN)')
plt.xlabel('Coefficient Value')
plt.ylabel('Feature')
plt.show()

# Evaluate the model on the test set
y0_pred_logit = logit_model_sklearn.predict(X0_test_scaled)

print('\nLogistic Regression Model Evaluation (on test set):')
print(f'Accuracy: {accuracy_score(y0_test, y0_pred_logit):.2f}')
print('Classification Report:')
print(classification_report(y0_test, y0_pred_logit))
print('Confusion Matrix:')
print(confusion_matrix(y0_test, y0_pred_logit))

## Feature Importance for PARTY_WIN from Decision Tree Classifier

In [ ]:
# Define the features (X) and the target variable (y)
# Exclude the target variables themselves from the features
features_for_dtc = [col for col in VOTE_DF.columns if col not in [
    'GEOID', 'Male_total', 'Female_total', 'PARTY_WIN', 'PARTY_LEAD']]

# Define the features (X) and the target variable (y)
X1 = VOTE_DF[features_for_dtc]
y1 = VOTE_DF['PARTY_WIN']

# Split data into training and testing sets
X1_train, X1_test, y1_train, y1_test = train_test_split(
    X1, y1, test_size=0.2, random_state=1)

# Initialize and train the Decision Tree Regressor model
dtc_model = DecisionTreeClassifier(
    random_state=1,
    max_depth=5,
    min_samples_split=20,
    min_samples_leaf=10)

dtc_model.fit(X1_train, y1_train)

# Make predictions on the test set
y1_pred_dtc = dtc_model.predict(X1_test)

# Evaluate the model
mse = mean_squared_error(y1_test, y1_pred_dtc)
rmse = np.sqrt(mse) # Calculate RMSE manually
mae = mean_absolute_error(y1_test, y1_pred_dtc)
r2 = r2_score(y1_test, y1_pred_dtc)

print('Decision Tree Regressor Model Evaluation (on test set):')
print(f'Mean Squared Error (MSE): {mse:.2f}')
print(f'Root Mean Squared Error (RMSE): {rmse:.2f}')
print(f'Mean Absolute Error (MAE): {mae:.2f}')
print(f'R-squared (R2): {r2:.2f}')

# Plot feature importances from the trained model
print('\nFeature Importance from Decision Tree Regressor:')
dtc_feature_importance = pd.Series(dtc_model.feature_importances_, index=features_for_dtc)

# Sort and print feature importances
print(dtc_feature_importance.sort_values(ascending=False).head(15))

# Plot feature importances
plt.figure(figsize=(10, 8)) # Adjusted figure size
dtc_feature_importance.sort_values().plot(kind='barh')
plt.title('Feature Importance from Decision Tree Regressor(WIN)')
plt.xlabel('Importance Score')
plt.ylabel('Feature')
plt.show()

## Feature Importance for PARTY_LEAD from Decision Tree Regressor

In [ ]:
X2 = VOTE_DF[features_for_dtc]
y2 = VOTE_DF['PARTY_LEAD']

# Split data into training and testing sets
X2_train, X2_test, y2_train, y2_test = train_test_split(
    X2, y2, test_size=0.2, random_state=1)

# Initialize and train the Decision Tree Regressor model
dtr_model = DecisionTreeRegressor(
    random_state=1,
    max_depth=5,
    min_samples_split=20,
    min_samples_leaf=10)

dtr_model.fit(X2_train, y2_train)

# Make predictions on the test set
y2_pred_dtr = dtr_model.predict(X2_test)

# Evaluate the model
mse = mean_squared_error(y2_test, y2_pred_dtr)
rmse = np.sqrt(mse) # Calculate RMSE manually
mae = mean_absolute_error(y2_test, y2_pred_dtr)
r2 = r2_score(y2_test, y2_pred_dtr)

print('Decision Tree Regressor Model Evaluation (on test set):')
print(f'Mean Squared Error (MSE): {mse:.2f}')
print(f'Root Mean Squared Error (RMSE): {rmse:.2f}')
print(f'Mean Absolute Error (MAE): {mae:.2f}')
print(f'R-squared (R2): {r2:.2f}')

# Get and plot feature importances from the trained model
print('\nFeature Importance from Decision Tree Regressor:')
dtr_feature_importance = pd.Series(dtr_model.feature_importances_, index=features_for_dtc)

# Sort and print feature importances
print(dtr_feature_importance.sort_values(ascending=False).head(20))

# Plot feature importances
plt.figure(figsize=(10, 8)) # Adjusted figure size
dtr_feature_importance.sort_values().plot(kind='barh')
plt.title('Feature Importance from Decision Tree Regressor(LEAD)')
plt.xlabel('Importance Score')
plt.ylabel('Feature')
plt.show()

## Feature Importance for PARTY_WIN from Random Forest

In [ ]:
# Select features for the Random Forest model
# We can use the same set of features that worked for the logistic regression.
features_for_rf = features_for_logit

X3 = VOTE_DF[features_for_rf]
y3 = VOTE_DF['PARTY_WIN']

# Split data into training and testing sets
X3_train, X3_test, y3_train, y3_test = train_test_split(
    X3, y3, test_size=0.2, random_state=1, stratify=y3)

# Initialize and train the Random Forest Classifier
# Use a reasonable number of estimators (n_estimators) and a random state for reproducibility
rf_model = RandomForestClassifier(
    n_estimators=300,
    max_depth=None,        # control/avoid overfitting
    min_samples_split=10,  # avoid tiny splits
    min_samples_leaf=5,    # smoother trees
    random_state=1,
    class_weight='balanced')
rf_model.fit(X3_train, y3_train)

# Get feature importances from the trained model
rf_feature_importance = pd.Series(
    rf_model.feature_importances_, index=features_for_rf)

# Sort and print feature importances
print('Feature Importance from Random Forest:')
print(rf_feature_importance.sort_values(ascending=False))

# Plot feature importances
plt.figure(figsize=(10, 6))
rf_feature_importance.sort_values().plot(kind='barh')
plt.title('Feature Importance from Random Forest(WIN)')
plt.xlabel('Importance Score (Mean Decrease in Impurity)')
plt.ylabel('Feature')
plt.show()

# Evaluate the model on the test set
y3_pred_rf = rf_model.predict(X3_test)

print('\nRandom Forest Model Evaluation (on test set):')
print(f'Accuracy: {accuracy_score(y3_test, y3_pred_rf):.2f}')
print('Classification Report:')
print(classification_report(y3_test, y3_pred_rf))
print('Confusion Matrix:')
print(confusion_matrix(y3_test, y3_pred_rf))

### Define permutation importance function (with optional cross-validation)

In [ ]:
# Compute permutation importance (PI) or cross-validated PI (CV-PI)
def get_PI(model, X3, y3, cv=False, n_splits=5, n_repeats=10, random_state=1):
    '''
    Parameters
    ----------
    model : estimator
        Trained model (must support predict).
    X : DataFrame
        Features used for prediction.
    y : Series or array-like
        Target values.
    cv : bool, default=False
        If True, performs cross-validated permutation importance.
    n_splits : int, default=5
        Number of CV folds (only used if cv=True).
    n_repeats : int, default=10
        Number of shuffles for permutation importance.
    random_state : int, default=1
        Random seed for reproducibility.

    Returns
    -------
    importance_df : DataFrame
        Feature importances sorted by mean decrease in score.
    '''

    if not cv:
# Standard PI on a single fitted model
        result = permutation_importance(model, X3, y3,
                                        n_repeats=n_repeats,
                                        random_state=random_state,
                                        n_jobs=-1)
        importance_df = pd.DataFrame({
            'Feature': X3.columns,
            'Importance Mean': result.importances_mean,
            'Importance Std': result.importances_std
        }).sort_values(by='Importance Mean', ascending=False)

    else:
# Cross-validated PI
        skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
        importances = []

        for train_idx, test_idx in skf.split(X3, y3):
            X3_train, X3_test = X3.iloc[train_idx], X3.iloc[test_idx]
            y3_train, y3_test = y3.iloc[train_idx], y3.iloc[test_idx]

            model.fit(X3_train, y3_train)
            result = permutation_importance(model, X3_test, y3_test,
                                            n_repeats=n_repeats,
                                            random_state=random_state,
                                            n_jobs=-1)
            importances.append(result.importances_mean)

        mean_importances = np.mean(importances, axis=0)
        std_importances = np.std(importances, axis=0)

        importance_df = pd.DataFrame({
            'Feature': X3.columns,
            'Importance Mean': mean_importances,
            'Importance Std': std_importances
        }).sort_values(by='Importance Mean', ascending=False)
    return importance_df

# Confirm
pd.set_option('display.max_rows', None)
pd.reset_option('display.float_format')

RF_PI = get_PI(rf_model, X3_test, y3_test, cv=False)
print(RF_PI)

# Plot permutation importances
plt.figure(figsize=(10, 6))
# Convert 'Importance Mean' to numeric before plotting
RF_PI['Importance Mean'] = pd.to_numeric(RF_PI['Importance Mean'])
RF_PI.sort_values(by='Importance Mean', ascending=True).plot(kind='barh')
plt.title('Permutation Importance from Random Forest(WIN)')
plt.xlabel('Importance Score (Importance decrease in Mean)')
plt.ylabel('Feature')
plt.show()

### Run RFECV with Random Forest to confirm best features

In [ ]:
# Utillize X, y, train, test from Logit (X0, y0)
# RFECV with Random Forest
rf = RandomForestClassifier(n_estimators=500, random_state=1, class_weight='balanced')

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
selector = RFECV(estimator=rf, step=1, cv=cv, scoring='accuracy', n_jobs=-1)
selector.fit(X0_train, y0_train)

# Best features
best_features = X0.columns[selector.support_].tolist()
print('Best feature subset:')
print(best_features)

# Retrain final model with best features
rf_ECV = RandomForestClassifier(n_estimators=500, random_state=1, class_weight='balanced')
rf_ECV.fit(X0_train[best_features], y0_train)
y_pred_ECV = rf_ECV.predict(X0_test[best_features])

print('\nFinal Model Evaluation with Best Features:')
print(f'Accuracy: {accuracy_score(y0_test, y_pred_ECV):.4f}')
print('Classification Report:')
print(classification_report(y0_test, y_pred_ECV))
print('Confusion Matrix:')
print(confusion_matrix(y0_test, y_pred_ECV))

## Feature importance for PARTY_LEAD from Lasso Regression after Cross-validate Alpha

In [ ]:
# Define features and exclude target variables
features_for_lasso_cv = [col for col in VOTE_DF.columns if col not in ['GEOID', 'PARTY_WIN', 'PARTY_LEAD']]
X4 = VOTE_DF[features_for_lasso_cv]
y4 = VOTE_DF['PARTY_LEAD']

# Split into train and test sets
X4_train, X4_test, y4_train, y4_test = train_test_split(
    X4, y4, test_size=0.2, random_state=1)

# Standardize the features
scaler = StandardScaler()
X4_train_scaled = scaler.fit_transform(X4_train)
X4_test_scaled = scaler.transform(X4_test)

# Let LassoCV automatically generates an alpha grid to test
lasso_cv_model = LassoCV(cv=5, random_state=1, max_iter=10000)
lasso_cv_model.fit(X4_train_scaled, y4_train)

# Confirm the optimal alpha found by LassoCV
optimal_alpha = lasso_cv_model.alpha_
print(f'Optimal alpha found by LassoCV: {optimal_alpha:.4f}')
print('')
# Plot the MSE as a function of alpha
mse_path = lasso_cv_model.mse_path_
alphas = lasso_cv_model.alphas_

plt.figure(figsize=(10, 6))
plt.plot(alphas, mse_path, linestyle='-', marker='o')
plt.xscale('log') # Often useful to plot alpha on a log scale
plt.xlabel('Alpha')
plt.ylabel('Mean Squared Error (across folds)')
plt.title('Mean Squared Error vs. Alpha during Cross-validation')
plt.axvline(optimal_alpha, color='red', linestyle='--', label=f'Optimal Alpha = {optimal_alpha:.4f}')
plt.legend()
plt.grid(True)
plt.show()

# List the coefficients with the optimal alpha
print('\nFeature Importance from LassoCV (CV = 0.0002):')
feature_importance_lasso_cv = pd.Series(lasso_cv_model.coef_, index=features_for_lasso_cv)
print(feature_importance_lasso_cv.sort_values(ascending=False))

# Plot feature importances with optimal alpha
plt.figure(figsize=(10, 10))
feature_importance_lasso_cv.sort_values().plot(kind='barh')
plt.title(f'Feature Importance from Lasso Regression(LEAD) with Optimal Alpha = {optimal_alpha:.4f}')
plt.xlabel('Coefficient Value')
plt.ylabel('Feature')
plt.show()

# Evaluate the final Lasso model with the optimal alpha on the test set
y4_pred_lasso_cv = lasso_cv_model.predict(X4_test_scaled)

print('\nLasso Regression Model Evaluation (with Optimal Alpha):')
mse_test = mean_squared_error(y4_test, y4_pred_lasso_cv)
rmse_test = np.sqrt(mse_test)
r2_test = r2_score(y4_test, y4_pred_lasso_cv)

print(f'Mean Squared Error (MSE) on test set: {mse_test:.4f}')
print(f'Root Mean Squared Error (RMSE) on test set: {rmse_test:.4f}')
print(f'R-squared (R2) on test set: {r2_test:.4f}')

## Compare feature importance

In [ ]:
def normalize_series(s, keep_sign=True):
    '''Normalize feature importance to 0–1 scale, optionally keeping sign.'''
    s = s.fillna(0)
    if keep_sign:
        return s / s.abs().max()  # scale to -1..1, preserving sign
    else:
        scaler = MinMaxScaler()
        return pd.Series(scaler.fit_transform(s.values.reshape(-1, 1)).flatten(), index=s.index)

# Collect raw importances into a DataFrame
feature_importances = pd.DataFrame({
    'LogReg': logit_feature_importance,
    'DecTreeClass': dtc_feature_importance,
    'DecTreeReg': dtr_feature_importance,
    'RandomForest': rf_feature_importance,
    'RF_PI': RF_PI.set_index('Feature')['Importance Mean'],  # permutation importance
    'Lasso_LogReg': feature_importance_lasso_cv})

# Normalize each column (preserving signs)
for col in feature_importances.columns:
    if col in ['LogReg', 'Lasso_LogReg']:  # signed coefficients
        feature_importances[col] = normalize_series(feature_importances[col], keep_sign=True)
    else:  # tree-based importances are ≥ 0
        feature_importances[col] = normalize_series(feature_importances[col], keep_sign=False)

# Compute mean rank or average importance across models
feature_importances['Avg_Importance'] = feature_importances.abs().mean(axis=1)

# Sort by average importance
feature_importances = feature_importances.sort_values(by='Avg_Importance', ascending=False)

# Print the top features
print('\nTop 25 Features Across Models (normalized):')
display(feature_importances.round(4))

# RFECV Feature Selection for final model

In [ ]:
# Prepare Data, drop reference and overlap features
drop_features = [
    'Male_total', 'Female_total', # reference only
    '%AGE_MALE_YNG', '%AGE_MALE_MID', '%AGE_MALE_OLD', # overlap
    '%AGE_FEMALE_YNG', '%AGE_FEMALE_MID', '%AGE_FEMALE_OLD', # overlap
    '%RACE_NonWhite', '%RACE_BAO', '%RACE_LNHM'] # overlap

VOTE_FULL = VOTE_DF.drop(
    columns=drop_features,
    errors='ignore')

X = VOTE_FULL.drop(
    columns=['GEOID', 'PARTY_WIN', 'PARTY_LEAD'])
y = VOTE_FULL['PARTY_WIN']

# Train/test split
XF_train, XF_test, yF_train, yF_test = train_test_split(
    X, y, test_size=0.2, random_state=1, stratify=y)

# Recursive Feature Elimination with Cross-Validation
rf = RandomForestClassifier(
    n_estimators=400, random_state=1, class_weight='balanced')
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
selector = RFECV(
    estimator=rf, step=1, cv=cv, scoring='accuracy', n_jobs=-1)
selector.fit(XF_train, yF_train)

# Plot accuracy vs. number of features
n_features = np.arange(
    1, len(selector.cv_results_['mean_test_score']) + 1)

plt.figure(figsize=(8,6))
plt.plot(n_features, selector.cv_results_['mean_test_score'], marker='o')
plt.axhline(0.93, color='red', linestyle='--', label='93% threshold')
plt.axhline(max(selector.cv_results_['mean_test_score']), color='green', linestyle='--', label='Best Acc')
plt.xlabel('Number of Features Selected')
plt.ylabel('Cross-Validated Accuracy')
plt.title('Accuracy vs. Number of Features (RFECV)')
plt.legend()
plt.show()

# Retrain with Best Features
best_features = X.columns[
    selector.support_].tolist()
print('\nBest Feature Subset:')
print(best_features)

rf_final = RandomForestClassifier(
    n_estimators=400, random_state=1, class_weight='balanced')
rf_final.fit(
    XF_train[best_features], yF_train)
yF_pred = rf_final.predict(
    XF_test[best_features])

print('\nFinal Model Evaluation with Best Features:')
print(f'Accuracy: {accuracy_score(yF_test, yF_pred):.4f}')
print('Classification Report:')
print(classification_report(yF_test, yF_pred))
print('Confusion Matrix:')
print(confusion_matrix(yF_test, yF_pred))

# Plot Feature Importance
importances = rf_final.feature_importances_
feat_imp = pd.DataFrame({
    'Feature': best_features,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)

print('\nTop Features Driving Model Accuracy:')
display(feat_imp)

plt.figure(figsize=(8,6))
plt.barh(feat_imp['Feature'], feat_imp['Importance'], color='steelblue')
plt.gca().invert_yaxis()
plt.xlabel('Feature Importance (RF)')
plt.title('Key Demographic Predictors of Voting Patterns')
plt.show()

## Train FINAL MODEL

In [ ]:
# Select final features: From 'accuracy vs. number of features' plot, ideal number of features start at 8, 9, or 10, compare 8-10 features for accuracy and MANOVA scores (dropped HH_MARRIED due to correlation with REL_xxx_MAR)

final_features = [ # Drop '%AGE_FEMALE_YOUNG', '%REL_W_RELATIVES' for best results
    'GEOID', 'PARTY_WIN', 'PARTY_LEAD',
    '%RACE_White', '%RACE_Asian', '%Urban_pop', '%REL_S_SEX_MAR',
    '%REL_OP_SEX_MAR', '%OWN_HOME', '%REL_NON_REL', '%RACE_Black']
VOTE_FINAL = VOTE_DF[final_features]

# Set features for final model
X_final = [col for col in VOTE_FINAL.columns if col not in [
    'GEOID', 'PARTY_WIN', 'PARTY_LEAD']]
y_final = VOTE_FINAL['PARTY_WIN']

# Train/test split
XF_train, XF_test, yF_train, yF_test = train_test_split(
    VOTE_FINAL[X_final], y_final,
    test_size=0.2, random_state=1,
    stratify=y_final)

# Train Random Forest
rf_final = RandomForestClassifier(
    n_estimators=500,
    random_state=1,
    class_weight='balanced')
rf_final.fit(XF_train, yF_train)

# Evaluate
yF_pred = rf_final.predict(XF_test)

print('\nFinal Model Evaluation:')
print(f'Accuracy: {accuracy_score(yF_test, yF_pred):.4f}')
print('Classification Report:')
print(classification_report(yF_test, yF_pred))
print('Confusion Matrix:')
print(confusion_matrix(yF_test, yF_pred))

## Use MANOVA to assess whether multiple features jointly differ between Democrats and Republicans

In [ ]:
maov = MANOVA(endog=VOTE_FINAL[X_final], exog=VOTE_FINAL[[y_final.name]])
print(maov.mv_test())

## Run CV PI

In [ ]:
# Cross-validate Permutation Importance
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
importances = []

# Pass feature data to skf.split
for train_fcv, test_fcv in skf.split(VOTE_FINAL[X_final], y_final):
    XF_train, XF_test = VOTE_FINAL[X_final].iloc[train_fcv], VOTE_FINAL[X_final].iloc[test_fcv]
    yF_train, yF_test = y_final.iloc[train_fcv], y_final.iloc[test_fcv]

    rf_final.fit(XF_train, yF_train)
    result = permutation_importance(
        rf_final, XF_test, yF_test,
        n_repeats=10, random_state=1, n_jobs=-1)
    importances.append(result.importances_mean)

mean_importances = np.mean(importances, axis=0)
std_importances = np.std(importances, axis=0)

# Build PI DF
pi_df = pd.DataFrame({
    'Feature': X_final, # Use X_final for feature names
    'Importance Mean': mean_importances.round(4),
    'Importance Std': std_importances.round(4)
}).sort_values(by='Importance Mean', ascending=False)

# Confirm
print('\nCross-validated Permutation Importance:')
print(pi_df)

# Plot
plt.figure(figsize=(10, 6))
pi_df.set_index('Feature')['Importance Mean'].sort_values().plot(kind='barh')
plt.title('Final Model - CV Permutation Importance')
plt.xlabel('Mean Importance (± CV variation)')
plt.ylabel('Feature')
plt.show()

# Profile counties with **extremely high PARTY_LEAD** (-30 > LEAD > +30)  

Do the demographics of partisan counties match final feature importance?

In [ ]:
VOTE_FINAL.to_csv('VOTE_FINAL.csv', index=False)

## 1. Equal Cutoff Strongholds (±0.5)

In [ ]:
def profile_group(dataframe, name, features):
    '''Calculates the mean of specified features for a group and returns a Series.'''
# Ensure only numeric columns in features are selected for mean calculation
    numeric_features = dataframe[features].select_dtypes(include=np.number).columns.tolist()
    profile = dataframe[numeric_features].mean()
    profile.name = name
    return profile

# Set cutoff value to 0.50
cutoff_val = 0.50

# Use _FIN to allow R and D access to all variables
extreme_counties = VOTE_FULL[np.abs(VOTE_FULL['PARTY_LEAD']) > cutoff_val]

# Republican strongholds
extreme_R = extreme_counties[extreme_counties['PARTY_LEAD'] < -cutoff_val]

# Democratic strongholds
extreme_D = extreme_counties[extreme_counties['PARTY_LEAD'] > cutoff_val]

print(f'Republican strongholds (cutoff -{cutoff_val}):', extreme_R.shape[0])
print(f'Democratic strongholds (cutoff +{cutoff_val}):', extreme_D.shape[0])

# Select demographic features only (drop outcomes and GEOID)
demo_features = [col for col in VOTE_FULL.columns if col not in ['PARTY_WIN', 'PARTY_LEAD', 'GEOID']]

# Profiles for cutoff-based groups
cutoff_profiles_combined = pd.concat([
    profile_group(extreme_R, 'R_characteristics', demo_features),
    profile_group(extreme_D, 'D_characteristics', demo_features),
], axis=1)

# Add absolute difference column
cutoff_profiles_combined['Abs_Diff'] = np.abs(cutoff_profiles_combined['R_characteristics'] - cutoff_profiles_combined['D_characteristics'])

print(f'\n=== Cutoff-based Stronghold Profiles (>{cutoff_val} Party Lead) ===')
print(cutoff_profiles_combined.sort_values(by='Abs_Diff', ascending=False))

## 2. Balanced Strongholds (Top/Bottom 10% quantiles)

In [ ]:
# Set cutoff value
lower_10 = VOTE_FULL['PARTY_LEAD'].quantile(0.10)   # bottom 10% cutoff
upper_10 = VOTE_FULL['PARTY_LEAD'].quantile(0.90)   # top 10% cutoff

R_stnghd_bal = VOTE_FULL[VOTE_FULL['PARTY_LEAD'] <= lower_10].copy()
D_stnghd_bal = VOTE_FULL[VOTE_FULL['PARTY_LEAD'] >= upper_10].copy()

print(f'Republican strongholds (quantile-based): {len(R_stnghd_bal)} counties (<= {lower_10:.2f})')
print(f'Democratic strongholds (quantile-based): {len(D_stnghd_bal)} counties (>= {upper_10:.2f})')

# Select demographic features only (drop outcomes and GEOID)
demo_features = [col for col in VOTE_FULL.columns if col not in [
    'GEOID', 'PARTY_WIN', 'PARTY_LEAD']]

# Profiles for quantile-based groups
balanced_profiles = pd.concat([
    profile_group(R_stnghd_bal, 'R_characteristics', demo_features),
    profile_group(D_stnghd_bal, 'D_characteristics', demo_features),
], axis=1)

# Add absolute difference column
balanced_profiles['Abs_Diff'] = np.abs(balanced_profiles['R_characteristics'] - balanced_profiles['D_characteristics'])

print('\n=== Quantile-based Stronghold Profiles (Top/Bottom 10%) ===')
print(balanced_profiles.sort_values(by='Abs_Diff', ascending=False))

## Comparison Table

In [ ]:
# Build combined comparison table

# Profiles for cutoff-based groups
# Select demographic features only (drop outcomes and GEOID)
demo_features = [col for col in VOTE_FULL.columns if col not in ['PARTY_WIN', 'PARTY_LEAD', 'GEOID']]

cutoff_profiles = pd.concat([
    profile_group(extreme_R, 'R_cutoff', demo_features),
    profile_group(extreme_D, 'D_cutoff', demo_features)
], axis=1)

# Profiles for quantile-based groups
# Select demographic features only (drop outcomes and GEOID)
demo_features = [col for col in VOTE_FULL.columns if col not in ['PARTY_WIN', 'PARTY_LEAD', 'GEOID']]

balanced_profiles = pd.concat([
    profile_group(R_stnghd_bal, 'R_quantile', demo_features),
    profile_group(D_stnghd_bal, 'D_quantile', demo_features)
], axis=1)

# Combine both into one big table
comparison_table = pd.concat([cutoff_profiles, balanced_profiles], axis=1)

# Add difference columns (D – R) for clarity of spread
comparison_table['Diff_cutoff'] = comparison_table['D_cutoff'] - comparison_table['R_cutoff']
comparison_table['Diff_quantile'] = comparison_table['D_quantile'] - comparison_table['R_quantile']

comparison_table = comparison_table.round(2)

# Confirm
print('\n=== Combined Stronghold Profiles (Cutoff vs Quantile) ===')
display(comparison_table.sort_values(by='R_cutoff', ascending=False))

## Locations of Extremes

In [ ]:
# Pad GEOIDs with a leading zero if length is less than 5 and extract state FIPS
R_geoids = [f'{int(geo):05d}' if len(geo) < 5 else geo for geo in extreme_R['GEOID'].unique()]
D_geoids = [f'{int(geo):05d}' if len(geo) < 5 else geo for geo in extreme_D['GEOID'].unique()]

R_fips = [geo[:2] for geo in R_geoids]
D_fips = [geo[:2] for geo in D_geoids]

R_counts = Counter(R_fips)
D_counts = Counter(D_fips)

print('\nCounts for counties above 50% party lead (Min 75-25% split):')
# Sort state_counts by 2-digit state FIPS)
sorted_R_counts = dict(sorted(R_counts.items()))
sorted_D_counts = dict(sorted(D_counts.items()))
print(sorted_R_counts)
print(sorted_D_counts)

# Results

## 1. Model Performance & Prediction Quality

In [ ]:
# Predictions: Use the subset of features that the model was trained on
yF_pred = rf_final.predict(XF_test)
y_pred_proba = rf_final.predict_proba(XF_test)[:, 1] if hasattr(
               rf_final, 'predict_proba') else None

# Collect metrics
performance = {
    'Accuracy': accuracy_score(yF_test, yF_pred),
    'Precision': precision_score(yF_test, yF_pred, zero_division=0),
    'Recall': recall_score(yF_test, yF_pred, zero_division=0),
    'F1 Score': f1_score(yF_test, yF_pred, zero_division=0)}

if y_pred_proba is not None:
    performance['ROC-AUC'] = roc_auc_score(yF_test, y_pred_proba)

# Create performance DataFrame
perf_df = pd.DataFrame(performance, index=['Final Model']).T
display(perf_df)

## 2. Feature Importance

In [ ]:
print('\nCross-validated Permutation Importance:')
print(pi_df)

# Plot
plt.figure(figsize=(10, 6))
pi_df.set_index('Feature')['Importance Mean'].sort_values().plot(kind='barh')
plt.title('Final Model - CV Permutation Importance')
plt.xlabel('Mean Importance (± CV variation)')
plt.ylabel('Feature')
plt.show()

END

##Read Minimum Wage Data  
Datafile manually compiled from  
state law https://www.dol.gov/agencies/whd/state/minimum-wage/history  
and local ordinances https://laborcenter.berkeley.edu/inventory-of-us-city-and-county-minimum-wage-ordinances/  
Many exceptions exist for number of employees, annual receipts, type of work, tip workers, etc. Highest wage selected for study.